# Projet INSEE Partie (3) - RÉGRESSION PROBIT BINAIRE - TRANSFRONTALIER MOBILITÉ TRANSFRONTALIÈRE

## Step 1 : Préparation de la base donnée

In [1]:
import pandas as pd

# Charger le fichier X
X = pd.read_csv('/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_READY/FINAL_AFTER_ARBITRAGES/X_final_clean.csv')

print("="*80)
print("STRUCTURE DU FICHIER X_final_clean.csv")
print("="*80)

print(f"\nDimensions : {X.shape[0]:,} lignes × {X.shape[1]} colonnes")

print("\n" + "="*80)
print("LISTE DES COLONNES")
print("="*80)
for i, col in enumerate(X.columns):
    print(f"  {i+1:>2}. {col}")

print("\n" + "="*80)
print("TYPES DE DONNÉES")
print("="*80)
print(X.dtypes)

print("\n" + "="*80)
print("APERÇU (5 premières lignes)")
print("="*80)
print(X.head())

print("\n" + "="*80)
print("STATISTIQUES")
print("="*80)
print(X.describe().T)

print("\n" + "="*80)
print("VALEURS MANQUANTES")
print("="*80)
print(X.isnull().sum()[X.isnull().sum() > 0])

STRUCTURE DU FICHIER X_final_clean.csv

Dimensions : 494,483 lignes × 81 colonnes

LISTE DES COLONNES
   1. AGEREV
   2. AGEREV_sq
   3. GS_1
   4. GS_2
   5. GS_3
   6. GS_4
   7. GS_5
   8. GS_6
   9. EMPL_11
  10. EMPL_12
  11. EMPL_13
  12. EMPL_14
  13. EMPL_15
  14. EMPL_16
  15. EMPL_21
  16. EMPL_22
  17. EMPL_23
  18. INATC_1
  19. INATC_2
  20. COUPLE_1
  21. COUPLE_2
  22. NENFR_0
  23. NENFR_1
  24. NENFR_2
  25. NENFR_3
  26. NENFR_4
  27. NENFR_Z
  28. NA5_AZ
  29. NA5_BE
  30. NA5_FZ
  31. NA5_GU
  32. NA5_OQ
  33. DIPL_1
  34. DIPL_11
  35. DIPL_12
  36. DIPL_13
  37. DIPL_14
  38. DIPL_15
  39. DIPL_16
  40. DIPL_17
  41. DIPL_18
  42. DIPL_19
  43. DIPL_2
  44. DIPL_3
  45. ETUD_1
  46. ETUD_2
  47. SANI_0
  48. SANI_1
  49. SANI_2
  50. SANI_X
  51. SEXE_1
  52. SEXE_2
  53. TP_1
  54. TP_2
  55. DEPT_10
  56. DEPT_51
  57. DEPT_52
  58. DEPT_54
  59. DEPT_55
  60. DEPT_57
  61. DEPT_67
  62. DEPT_68
  63. DEPT_88
  64. STOCD_10.0
  65. STOCD_21.0
  66. STOCD_22.0
  

# Choix des catégories de références + check vif

In [2]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ============================================
# CHARGER DONNÉES
# ============================================
X = pd.read_csv('/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_READY/FINAL_AFTER_ARBITRAGES/X_final_clean.csv')
y = pd.read_csv('/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_READY/FINAL_AFTER_ARBITRAGES/y_final.csv')

df = X.copy()
df['Y'] = y.values.ravel() if len(y.shape) > 1 else y.values

print("="*80)
print("DIAGNOSTIC PRÉ-PROBIT : 17 VARIABLES CONCEPTUELLES")
print("="*80)
print(f"\nBase : {len(df):,} obs | Transfrontaliers : {df['Y'].sum():,} ({df['Y'].mean()*100:.2f}%)")

# ============================================
# 1. MAPPING DUMMIES → VARIABLES CONCEPTUELLES
# ============================================
print("\n" + "="*80)
print("1. STRUCTURE : 81 DUMMIES → 17 VARIABLES CONCEPTUELLES")
print("="*80)

# Identifier les préfixes (variables conceptuelles)
def get_prefix(col):
    if col in ['AGEREV', 'AGEREV_sq']:
        return 'AGEREV'
    elif col.startswith('DNAI_'):
        return 'DNAI'
    elif '_' in col:
        return col.rsplit('_', 1)[0]
    else:
        return col

# Regrouper colonnes par variable conceptuelle
var_mapping = {}
for col in X.columns:
    prefix = get_prefix(col)
    if prefix not in var_mapping:
        var_mapping[prefix] = []
    var_mapping[prefix].append(col)

print(f"\n{'Variable conceptuelle':<15} {'Nb dummies':>12} {'Colonnes'}")
print("-"*80)
for var, cols in sorted(var_mapping.items()):
    cols_str = ', '.join(cols[:3])
    if len(cols) > 3:
        cols_str += f" ... (+{len(cols)-3})"
    print(f"{var:<15} {len(cols):>12} {cols_str}")

# ============================================
# 2. DÉTAIL PAR VARIABLE : EFFECTIFS + TAUX TRANSFRONTALIER
# ============================================
print("\n" + "="*80)
print("2. MODALITÉS PAR VARIABLE (effectifs + taux transfrontalier)")
print("="*80)
print("→ Pour choisir la CATÉGORIE DE RÉFÉRENCE du Probit\n")

ref_suggestions = {}

for var, cols in sorted(var_mapping.items()):
    print(f"\n{'─'*60}")
    print(f"📊 {var} ({len(cols)} modalités)")
    print(f"{'─'*60}")
    
    if var == 'AGEREV':
        # Variable continue
        print(f"  Type: CONTINUE")
        print(f"  Moyenne: {X['AGEREV'].mean():.1f} ans | Écart-type: {X['AGEREV'].std():.1f}")
        print(f"  Min: {X['AGEREV'].min()} | Max: {X['AGEREV'].max()}")
        print(f"  → Pas de catégorie de référence (variable continue)")
        ref_suggestions[var] = None
        continue
    
    print(f"  {'Modalité':<20} {'Effectif':>12} {'%':>8} {'Taux transf.':>14} {'Suggestion'}")
    print(f"  {'-'*70}")
    
    modal_stats = []
    for col in cols:
        modalite = col.replace(var + '_', '')
        effectif = X[col].sum()
        pct = effectif / len(X) * 100
        taux_transf = df[df[col] == 1]['Y'].mean() * 100 if effectif > 0 else 0
        modal_stats.append({
            'col': col,
            'modalite': modalite,
            'effectif': effectif,
            'pct': pct,
            'taux': taux_transf
        })
    
    # Trier par effectif décroissant
    modal_stats = sorted(modal_stats, key=lambda x: x['effectif'], reverse=True)
    
    # La plus fréquente = suggestion de référence
    ref_col = modal_stats[0]['col']
    ref_suggestions[var] = ref_col
    
    for i, m in enumerate(modal_stats):
        suggestion = "← REF (plus fréquente)" if i == 0 else ""
        print(f"  {m['modalite']:<20} {m['effectif']:>12,} {m['pct']:>7.1f}% {m['taux']:>13.1f}% {suggestion}")

# ============================================
# 3. VIF SUR VARIABLES CONCEPTUELLES
# ============================================
print("\n" + "="*80)
print("3. VIF - MULTICOLINÉARITÉ (sur 17 variables conceptuelles)")
print("="*80)
print("Méthode: Label encoding des catégorielles → VIF")
print("Seuils : VIF < 5 ✓ | 5-10 ⚠️ | ≥10 🔴\n")

# Reconstruire les variables conceptuelles en label encoding
X_conceptual = pd.DataFrame()

for var, cols in var_mapping.items():
    if var == 'AGEREV':
        X_conceptual['AGEREV'] = X['AGEREV']
        # On n'inclut pas AGEREV_sq car colinéaire par construction
    else:
        # Reconstruire la variable catégorielle
        # Trouver quelle modalité = 1 pour chaque ligne
        temp = X[cols].idxmax(axis=1)
        # Encoder en numérique
        X_conceptual[var] = pd.factorize(temp)[0]

# Calculer VIF
print(f"Calcul VIF sur {len(X_conceptual.columns)} variables...")

vif_results = []
for i, col in enumerate(X_conceptual.columns):
    try:
        vif = variance_inflation_factor(X_conceptual.values.astype(float), i)
        vif_results.append({'Variable': col, 'VIF': vif})
    except Exception as e:
        vif_results.append({'Variable': col, 'VIF': np.nan})

vif_df = pd.DataFrame(vif_results).sort_values('VIF', ascending=False)

print(f"\n{'Variable':<15} {'VIF':>10} {'Statut'}")
print("-"*40)
for _, row in vif_df.iterrows():
    vif = row['VIF']
    if pd.isna(vif) or np.isinf(vif):
        statut = "❌ Erreur"
        vif_str = "∞"
    elif vif >= 10:
        statut = "🔴 Sévère"
        vif_str = f"{vif:.2f}"
    elif vif >= 5:
        statut = "⚠️ Modéré"
        vif_str = f"{vif:.2f}"
    else:
        statut = "✓ OK"
        vif_str = f"{vif:.2f}"
    print(f"{row['Variable']:<15} {vif_str:>10} {statut}")

vif_clean = vif_df[~vif_df['VIF'].isna() & ~np.isinf(vif_df['VIF'])]['VIF']
print("-"*40)
print(f"VIF moyen : {vif_clean.mean():.2f} | VIF max : {vif_clean.max():.2f}")

if vif_clean.max() < 5:
    print("\n✅ PAS DE MULTICOLINÉARITÉ PROBLÉMATIQUE")
else:
    print(f"\n⚠️ {(vif_clean >= 5).sum()} variable(s) avec VIF ≥ 5")

# ============================================
# 4. CORRÉLATIONS FORTES ENTRE VARIABLES CONCEPTUELLES
# ============================================
print("\n" + "="*80)
print("4. CORRÉLATIONS FORTES (|ρ| > 0.4)")
print("="*80)

corr = X_conceptual.corr(method='spearman')

pairs = []
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        r = corr.iloc[i, j]
        if abs(r) > 0.4:
            pairs.append((corr.columns[i], corr.columns[j], r))

if pairs:
    print(f"\n{'Variable 1':<15} {'Variable 2':<15} {'ρ Spearman':>12}")
    print("-"*45)
    for v1, v2, r in sorted(pairs, key=lambda x: abs(x[2]), reverse=True):
        print(f"{v1:<15} {v2:<15} {r:>12.3f}")
else:
    print("\n✅ Aucune corrélation |ρ| > 0.4")

# ============================================
# 5. RÉCAPITULATIF CATÉGORIES DE RÉFÉRENCE
# ============================================
print("\n" + "="*80)
print("5. CATÉGORIES DE RÉFÉRENCE SUGGÉRÉES (pour Probit)")
print("="*80)
print("Critère: modalité la plus fréquente (convention standard)\n")

print(f"{'Variable':<15} {'Référence suggérée':<30} {'Justification'}")
print("-"*70)

# Labels lisibles
labels = {
    'GS_5': 'Employés',
    'EMPL_16': 'CDI/Fonctionnaire',
    'INATC_1': 'Français',
    'COUPLE_1': 'En couple',
    'NENFR_0': '0 enfant',
    'NA5_GU': 'Commerce/Services',
    'DIPL_13': 'CAP/BEP',
    'ETUD_2': 'Non étudiant',
    'SANI_2': 'Salle de bain',
    'SEXE_1': 'Homme',
    'TP_1': 'Temps complet',
    'DEPT_57': 'Moselle',
    'STOCD_10.0': 'Propriétaire',
    'VOIT_2.0': '2 voitures',
    'TYPL_1.0': 'Maison',
    'DNAI_NEGRANDEST': 'Né Grand Est'
}

for var, ref_col in ref_suggestions.items():
    if ref_col is None:
        print(f"{var:<15} {'(continue)':<30} {'Pas de référence'}")
    else:
        label = labels.get(ref_col, ref_col.replace(var + '_', ''))
        effectif = X[ref_col].sum()
        pct = effectif / len(X) * 100
        print(f"{var:<15} {label:<30} {pct:.1f}% des obs")

print("\n" + "="*80)
print("FIN DU DIAGNOSTIC")
print("="*80)

DIAGNOSTIC PRÉ-PROBIT : 17 VARIABLES CONCEPTUELLES

Base : 494,483 obs | Transfrontaliers : 44,264 (8.95%)

1. STRUCTURE : 81 DUMMIES → 17 VARIABLES CONCEPTUELLES

Variable conceptuelle   Nb dummies Colonnes
--------------------------------------------------------------------------------
AGEREV                     2 AGEREV, AGEREV_sq
COUPLE                     2 COUPLE_1, COUPLE_2
DEPT                       9 DEPT_10, DEPT_51, DEPT_52 ... (+6)
DIPL                      12 DIPL_1, DIPL_11, DIPL_12 ... (+9)
DNAI                       3 DNAI_NEAUTREFR, DNAI_NEETRANGER, DNAI_NEGRANDEST
EMPL                       9 EMPL_11, EMPL_12, EMPL_13 ... (+6)
ETUD                       2 ETUD_1, ETUD_2
GS                         6 GS_1, GS_2, GS_3 ... (+3)
INATC                      2 INATC_1, INATC_2
NA5                        5 NA5_AZ, NA5_BE, NA5_FZ ... (+2)
NENFR                      6 NENFR_0, NENFR_1, NENFR_2 ... (+3)
SANI                       4 SANI_0, SANI_1, SANI_2 ... (+1)
SEXE            

# SYNTHÈSE EXHAUSTIVE DES VARIABLES DU MODÈLE PROBIT SIMPLE

## Projet INSEE - Mobilité Transfrontalière Grand Est

**Base de données** : 494,483 observations | Transfrontaliers : 44,264 (8.95%)

---

# 1. AGEREV — Âge en années révolues

**Source** : INDCVI (Fichier Individus)

| Propriété             | Valeur |
|-----------            |--------|
| **Type**              | Continue |
| **Description INSEE** | Âge en années révolues détaillé (âge au dernier anniversaire) |
| **Plage**             | 0 à 120 ans |
| **Dans le modèle**    | 15-101 ans (actifs occupés uniquement) |

**Statistiques dans la BDD** :
- Moyenne : 41.5 ans
- Écart-type : 12.5 ans
- Min : 15 ans | Max : 101 ans

**Spécification probit** : 
- `AGEREV` (effet linéaire)
- `AGEREV_sq` (effet quadratique) → Effet en U inversé confirmé par Elastic Net

**Pas de catégorie de référence** (variable continue)

---

# 2. COUPLE — Déclaration de vie en couple

**Source** : INDCVI (Fichier Individus)

| Code  | Libellé INSEE                    | Effectif | % Base | Taux transf. |
|------ |---------------                   |----------|--------|--------------|
| **1** | A déclaré vivre en couple        | 325,880  | 65.9%  | 9.7% |
| 2     | A déclaré ne pas vivre en couple | 168,603  | 34.1%  | 7.5% |

**Catégorie de référence suggérée** : `COUPLE_1` (En couple) — 65.9% des observations

**Note** : Variable déclarative, pas de distinction marié/pacsé/concubinage ici (voir STAT_CONJ si nécessaire)

---

# 3. DEPT — Département du lieu de résidence

**Source** : INDCVI + MOBPRO

| Code   | Département        | Effectif | % Base | Taux transf. | Interprétation |
|------  |-------------       |----------|--------|--------------|----------------|
| **67** | Bas-Rhin           | 118,398  | 23.9%  | 4.6% | Strasbourg, éloigné frontière |
| 57     | Moselle            | 98,869   | 20.0%  | **21.6%** | Proximité Luxembourg |
| 68     | Haut-Rhin          | 73,587   | 14.9%  | **14.1%** | Proximité Suisse/Allemagne|
| 54     | Meurthe-et-Moselle | 64,924   | 13.1%  | 9.7% | Proximité Luxembourg |
| 51     | Marne              | 53,239   | 10.8%  | 0.1% | Éloigné des frontières |
| 88     | Vosges             | 29,098   | 5.9%   | 0.2% | Relief montagneux |
| 10     | Aube               | 27,234   | 5.5%   | 0.1% | Éloigné des frontières |
| 55     | Meuse              | 14,885   | 3.0%   | 4.7% | Entre Metz et frontière |
| 52     | Haute-Marne        | 14,249   | 2.9%   | 0.1% | Éloigné des frontières |

**Catégorie de référence suggérée** : `DEPT_67` (Bas-Rhin) — 23.9% des observations

**Attention** : Le diagnostic suggère DEPT_67 (plus fréquent), mais économétriquement on pourrait préférer un département "neutre" éloigné des frontières (ex: DEPT_51 Marne) pour que tous les coefficients des départements frontaliers soient positifs et comparables.

## Choix final : alternative selectionnée pour DEPT
Utiliser DEPT_51 (Marne) comme référence pour avoir tous les coefficients des départements frontaliers en positif, ce qui facilite l'interprétation.

---

# 4. DIPL — Diplôme le plus élevé

**Source** : INDCVI (Fichier Individus)

| Code | Libellé INSEE complet                       | Effectif | % Base | Taux transf. |
|------|----------------------                       |----------|--------|--------------|
| 01 | Pas de scolarité ou arrêt avant fin primaire        | 3,431  | 0.7%  | 5.4% |
| 02 | Aucun diplôme, scolarité fin primaire/avant collège | 10,348 | 2.1%  | 8.1% |
| 03 | Aucun diplôme, scolarité jusqu'à fin collège ou +   | 26,638 | 5.4%  | 9.1% |
| 11 | CEP (certificat d'études primaires)                 | 4,062  | 0.8%  | 12.0% |
| 12 | BEPC, brevet élémentaire, DNB                       | 16,298 | 3.3%  | 8.1% |
| **13** | **CAP, BEP ou diplôme équivalent**              | 117,859|**23.8%**| 9.6% |
| 14 | Bac général/techno, brevet supérieur, DAEU          | 50,473 | 10.2% | 7.8% |
| 15 | Bac pro, brevet professionnel/technicien            | 56,154 | 11.4% | 8.7% |
| 16 | BTS, DUT, Deug, Deust (Bac+2)                       | 75,404 | 15.2% | 7.9% |
| 17 | Licence, licence pro, maîtrise (Bac+3/4)            | 63,260 | 12.8% | 8.3% |
| 18 | Master, DEA, DESS, grande école (Bac+5)             | 65,128 | 13.2% | **10.7%** |
| 19 | Doctorat de recherche (hors santé)                  | 5,428  | 1.1%  | **11.2%** |

**Catégorie de référence suggérée** : `DIPL_13` (CAP/BEP) — 23.8% des observations

**Interprétation** : Les très diplômés (18, 19) et les CEP (11) ont les taux transfrontaliers les plus élevés. L'effet diplôme n'est pas linéaire.

---

# 5. DNAI — Département de naissance (recodé)

**Source** : INDCVI — Variable DNAI recodée en 3 modalités

| Code recodé    | Description                        | Effectif | % Base | Taux transf. |
|-------------   |-------------                       |----------|--------|--------------|
| **NEGRANDEST** | Né dans le Grand Est            | 360,009 | 72.8% | 8.1% |
| NEAUTREFR      | Né ailleurs en France           | 71,270 | 14.4% | 5.8% |
| NEETRANGER     | Né à l'étranger (code 99 INSEE) | 63,204 | 12.8% | **17.5%** |

**Variable INSEE originale** : 
- Codes 08, 10, 51, 52, 54, 55, 57, 67, 68, 88 → NEGRANDEST
- Autres codes français → NEAUTREFR
- Code 99 → NEETRANGER

**Catégorie de référence suggérée** : `DNAI_NEGRANDEST` — 72.8% des observations

**Interprétation économique** : Les personnes nées à l'étranger ont 2× plus de chances d'être transfrontaliers (réseaux, expérience internationale, moins d'ancrage territorial).

---

# 6. EMPL — Condition d'emploi

**Source** : INDCVI + MOBPRO

| Code | Libellé INSEE                                 | Effectif | % Base | Taux transf. |
|------|---------------                                |----------|--------|--------------|
| 11 | Contrat d'apprentissage ou professionnalisation  | 14,172  | 2.9%    | 1.8% |
| 12 | Placés par agence d'intérim                      | 10,308  | 2.1%    | **10.8%** |
| 13 | Emplois aidés (CUI, avenir, etc.)                | 2,437   | 0.5%    | 1.3% |
| 14 | Stagiaires rémunérés en entreprise               | 1,350   | 0.3%    | 9.3% |
| 15 | Autres emplois à durée limitée (CDD, saisonnier) | 40,025  | 8.1%    | 5.7% |
| **16** | **CDI, titulaire fonction publique**         | 374,974 |**75.8%**| **10.4%** |
| 21 | Non salariés : Indépendants                      | 29,943  | 6.1%    | 2.6% |
| 22 | Non salariés : Employeurs                        | 20,770  | 4.2%    | 3.6% |
| 23 | Non salariés : Aides familiaux                   | 504     | 0.1%    | 3.8% |

**Catégorie de référence suggérée** : `EMPL_16` (CDI/Fonctionnaire) — 75.8% des observations

**Interprétation** : 
- Les CDI ont le taux transfrontalier le plus élevé (stabilité → acceptation déplacements)
- L'intérim (12) aussi élevé car souvent transfrontalier (agences spécialisées Luxembourg)
- Les indépendants (21) très faibles car activité ancrée localement

---

# 7. ETUD — Inscription dans un établissement d'enseignement

**Source** : INDCVI (Fichier Individus)

| Code  | Libellé INSEE         | Effectif | % Base | Taux transf. |
|------ |---------------        |----------|--------|--------------|
| 1     | Oui (inscrit)         | 25,247   | 5.1%   | 3.1% |
| **2** | **Non (non inscrit)** | 469,236  |**94.9%**| 9.3% |

**Catégorie de référence suggérée** : `ETUD_2` (Non étudiant) — 94.9% des observations

**Note** : Dans ta base, tu as filtré sur les actifs occupés, donc les "étudiants" ici sont des travailleurs-étudiants (alternance, formation continue).

---

# 8. GS — Groupe socioprofessionnel en 6 postes

**Source** : INDCVI + MOBPRO

| Code | Libellé INSEE                               | Effectif | % Base | Taux transf. |
|------|---------------                              |----------|--------|--------------|
| 1    | Agriculteurs exploitants                    | 5,990   | 1.2%    | **0.4%** |
| 2    | Artisans, commerçants, chefs d'entreprise   | 28,006  | 5.7%    | 3.2% |
| 3    | Cadres et professions intellect. supérieures| 80,429  | 16.3%   | 9.8% |
| 4    | Professions intermédiaires                  | 131,258 | 26.5%   | 8.2% |
|**5** | **Employés**                                | 133,175 |**26.9%**| 7.8% |
| 6    | Ouvriers                                    | 115,625 | 23.4%   | **12.3%** |

**Catégorie de référence suggérée** : `GS_5` (Employés) — 26.9% des observations

**Interprétation économique** :
- **Ouvriers (6)** : Taux le plus élevé (12.3%) → industrie transfrontalière (Luxembourg, Allemagne)
- **Agriculteurs (1)** : Taux quasi nul → activité terrienne par nature
- **Cadres (3)** : Taux élevé → différentiel salarial attractif Luxembourg/Suisse

---

# 9. INATC — Indicateur de nationalité condensé

**Source** : INDCVI + MOBPRO

| Code  | Libellé INSEE | Effectif | % Base | Taux transf. |
|------ |---------------|----------|--------|--------------|
| **1** | **Français**  | 458,904  |**92.8%**| 8.0% |
| 2     | Étrangers     | 35,579   | 7.2%   | **21.8%** |

**Catégorie de référence suggérée** : `INATC_1` (Français) — 92.8% des observations

**Interprétation** : Les étrangers résidant dans le Grand Est ont 2.7× plus de chances d'être transfrontaliers (proximité culturelle/linguistique avec pays voisin, réseaux, moins de contraintes administratives perçues).

---

# 10. NA5 — Activité économique regroupée en 5 postes

**Source** : INDCVI + MOBPRO (Nomenclature d'Activités Française)

| Code | Libellé INSEE                                | Effectif | % Base | Taux transf. |
|------|---------------                               |----------|--------|--------------|
| AZ | Agriculture, sylviculture et pêche              | 11,077 | 2.2%     | **0.7%** |
| BE | Industrie manufacturière, extractives et autres | 73,852 | 14.9%    | **12.9%** |
| FZ | Construction                                    | 32,768 | 6.6%     | **12.0%** |
| **GU** | **Commerce, transports et services divers** | 222,988 |**45.1%**| 11.4% |
| OQ | Administration publique, enseignement, santé    | 153,798 | 31.1%   | 3.5% |

**Catégorie de référence suggérée** : `NA5_GU` (Commerce/Services) — 45.1% des observations

**Interprétation économique** :
- **OQ (Administration)** : Taux très faible (3.5%) → emplois publics français par nature
- **AZ (Agriculture)** : Quasi nul → activité terrienne
- **BE (Industrie)** et **FZ (Construction)** : Très élevés → secteurs transfrontaliers typiques

---

# 11. NENFR — Nombre d'enfants de la famille (regroupé)

**Source** : INDCVI (Fichier Individus)

| Code  | Libellé INSEE                 | Effectif | % Base    | Taux transf. |
|------ |---------------                |----------|--------   |--------------|
| 0     | 0 enfant (couple sans enfant) | 116,349  | 23.5%     | 9.3% |
| **1** | **1 enfant**                  | 124,300  | **25.1%** | 9.3% |
| 2     | 2 enfants                     | 107,206  | 21.7%     | 9.5% |
| 3     | 3 enfants                     | 33,423   | 6.8%      | 8.4% |
| 4     | 4 enfants ou plus             | 10,672   | 2.2%      | 8.4% |
| Z     | Personne hors famille ou hors logement ordinaire | 102,533 | 20.7% | 7.8% |

**Catégorie de référence suggérée** : `NENFR_1` (1 enfant) — 25.1% des observations

**Choix final** : `NENFR_0` (0 enfant) car plus interprétable économiquement.

**Note** : La modalité "Z" correspond aux personnes vivant seules ou hors famille.

---

# 12. SANI — Installations sanitaires (France métropolitaine)

**Source** : INDCVI (Fichier Individus)

| Code  | Libellé INSEE                                | Effectif | % Base | Taux transf. |
|------ |---------------                               |----------|--------|--------------|
| 0     | Ni baignoire, ni douche                          | 650     | 0.1%  | 7.7% |
| 1     | Baignoire ou douche hors pièce réservée          | 15,297  | 3.1%  | **11.2%** |
| **2** | **Salle(s) de bains (avec douche ou baignoire)** | 475,443 |**96.1%**| 8.9% |
| X     | Hors logement ordinaire                          | 3,093   | 0.6%  | 0.5% |

**Catégorie de référence suggérée** : `SANI_2` (Salle de bains standard) — 96.1% des observations

**Interprétation** : Variable proxy du niveau de confort/standing du logement. Peut capter des effets de richesse.

---

# 13. SEXE — Sexe

**Source** : INDCVI + MOBPRO

| Code   | Libellé INSEE | Effectif | % Base | Taux transf. |
|------ |--------------  |----------|--------|--------------|
| **1** | **Hommes**     | 256,784  | **51.9%** | **10.5%** |
| 2     | Femmes         | 237,699  | 48.1%      | 7.2% |

**Catégorie de référence suggérée** : `SEXE_1` (Homme) — 51.9% des observations

**Interprétation économique** : Les hommes ont 1.5× plus de chances d'être transfrontaliers. Effet à explorer avec interactions (SEXE × NENFR, SEXE × EMPL) dans le modèle 2.

---

# 14. STOCD — Statut d'occupation détaillé du logement

**Source** : INDCVI + MOBPRO

| Code   | Libellé INSEE                       | Effectif | % Base | Taux transf. |
|------  |---------------                      |--------- |--------|--------------|
| 00     | Logement ordinaire inoccupé         | — | — | — |
| **10** | **Propriétaire**                    | 302,583  |**61.2%**| **10.4%** |
| 21     | Locataire logement vide non HLM     | 111,524  | 22.6%   | 8.3% |
| 22     | Locataire logement vide HLM         | 59,162   | 12.0%   | 4.0% |
| 23     | Locataire meublé ou chambre d'hôtel | 11,395   | 2.3%    | 6.4% |
| 30     | Logé gratuitement                   | 9,819    | 2.0%    | 5.3% |

**Catégorie de référence suggérée** : `STOCD_10` (Propriétaire) — 61.2% des observations

**Note** : Dans ta base, le code apparaît comme `STOCD_10.0` (float) → à vérifier/nettoyer.

**Interprétation** : Les propriétaires ont le taux le plus élevé (revenus → mobilité acceptée). Les locataires HLM ont le taux le plus faible.

---

# 15. TP — Temps de travail

**Source** : INDCVI + MOBPRO

| Code | Libellé INSEE   | Effectif | % Base    | Taux transf. |
|------|---------------  |----------|--------   |--------------|
|**1** |**Temps complet**| 411,499  | **83.2%** | 9.1% |
| 2    | Temps partiel   | 82,984   | 16.8%     | 8.3% |

**Catégorie de référence suggérée** : `TP_1` (Temps complet) — 83.2% des observations

**Note** : La modalité "Z" (Sans objet) a été exclue car tu travailles sur les actifs occupés.

---

# 16. TYPL — Type de logement

**Source** : INDCVI + MOBPRO

| Code | Libellé INSEE                       | Effectif | % Base | Taux transf. |
|------|---------------                      |----------|--------|--------------|
| **1** | **Maison**                         | 293,723  |**59.4%**| **10.1%** |
| 2     | Appartement                        | 198,316  | 40.1%   | 7.4% |
| 3     | Logement-foyer                     | 1,543    | 0.3%    | 4.2% |
| 4     | Chambre d'hôtel                    | 288      | 0.1%    | 4.9% |
| 5     | Habitation de fortune              | 278      | 0.1%    | 6.8% |
| 6     | Pièce indépendante (entrée propre) | 335      | 0.1%    | 3.9% |

**Catégorie de référence suggérée** : `TYPL_1` (Maison) — 59.4% des observations

**Note** : Dans ta base, le code apparaît comme `TYPL_1.0` (float) → à vérifier/nettoyer.

**Interprétation** : Les habitants de maisons sont plus souvent transfrontaliers (périurbain/rural frontalier vs. centre-ville).

---

# 17. VOIT — Nombre de voitures du ménage

**Source** : INDCVI + MOBPRO

| Code | Libellé INSEE           | Effectif | % Base | Taux transf. |
|------|---------------          |----------|--------|--------------|
| 0     | Aucune voiture         | 33,957   | 6.9%   | 4.2% |
| 1     | Une seule voiture      | 176,994  | 35.8%  | 7.7% |
| **2** | **Deux voitures**      | 217,963  |**44.1%**| 10.1% |
| 3     | Trois voitures ou plus | 65,569   | 13.3%  | **11.2%** |

**Catégorie de référence suggérée** : `VOIT_2` (2 voitures) — 44.1% des observations

**Note** : Dans ta base, le code apparaît comme `VOIT_2.0` (float) → à vérifier/nettoyer.

**Interprétation économique** : Effet monotone croissant → plus de voitures = plus de mobilité possible = plus de travail transfrontalier. Effet revenu + effet accès.

---

# TABLEAU RÉCAPITULATIF DES CATÉGORIES DE RÉFÉRENCE

| Variable | Référence suggérée| Libellé           | % Base | Justification |
|----------|-------------------|---------          |--------|---------------|
| AGEREV   | —                 | Continue          | —     | Pas de référence |
| COUPLE   | `COUPLE_1`        | En couple         | 65.9% | Plus fréquente |
| DEPT     | `DEPT_51`         | Marne          | 10.8% |Choix arbitraire "effet frontière"
| DIPL     | `DIPL_13`         | CAP/BEP           | 23.8% | Plus fréquente |
| DNAI     | `DNAI_NEGRANDEST` | Né Grand Est      | 72.8% | Plus fréquente |
| EMPL     | `EMPL_16`         | CDI/Fonctionnaire | 75.8% | Plus fréquente |
| ETUD     | `ETUD_2`          | Non étudiant      | 94.9% | Plus fréquente |
| GS       | `GS_5`            | Employés          | 26.9% | Plus fréquente |
| INATC    | `INATC_1`         | Français          | 92.8% | Plus fréquente |
| NA5      | `NA5_GU`          | Commerce/Services | 45.1% | Plus fréquente |
| NENFR    | `NENFR_0`         | 0 enfant     | 23.5% |Choix arbitraire "effet parentalité"
| SANI     | `SANI_2`          | Salle de bains    | 96.1% | Plus fréquente |
| SEXE     | `SEXE_1`          | Homme             | 51.9% | Plus fréquente |
| STOCD    | `STOCD_10`        | Propriétaire      | 61.2% | Plus fréquente |
| TP       | `TP_1`            | Temps complet     | 83.2% | Plus fréquente |
| TYPL     | `TYPL_1`          | Maison            | 59.4% | Plus fréquente |
| VOIT     | `VOIT_2`          | 1 voitures        | 35.8% | Choix arbitraire "médiane" |

---

# POINTS D'ATTENTION

## Variable NENFR
La modalité "Z" (hors famille) mélange :
- Personnes vivant seules
- Personnes en colocation hors famille
- Personnes en communauté

À interpréter avec prudence.

In [7]:
"""
================================================================================
PRÉPARATION DE LA BASE PROBIT - BDD_PROBIT
================================================================================
Création d'une base prête pour l'estimation probit en supprimant les catégories
de référence pour chaque variable catégorielle (éviter la multicolinéarité parfaite).

Catégories de référence retenues (17 variables conceptuelles) :
--------------------------------------------------------------------------------
Variable    | Référence      | Justification
--------------------------------------------------------------------------------
AGEREV      | (continue)     | Variable continue + terme quadratique
GS          | GS_5           | Employés - modalité la plus fréquente (26.9%)
EMPL        | EMPL_16        | CDI/Titulaire - modalité la plus fréquente (75.8%)
INATC       | INATC_1        | Français - modalité la plus fréquente (92.8%)
COUPLE      | COUPLE_1       | En couple - modalité la plus fréquente (65.9%)
NENFR       | NENFR_0        | 0 enfant - CHOIX UTILISATEUR (interprétation parentalité)
NA5         | NA5_GU         | Commerce/Services - modalité la plus fréquente (45.1%)
DIPL        | DIPL_13        | CAP/BEP - modalité la plus fréquente (23.8%)
ETUD        | ETUD_2         | Non étudiant - modalité la plus fréquente (94.9%)
SANI        | SANI_2         | Salle de bain standard - modalité la plus fréquente (96.1%)
SEXE        | SEXE_1         | Homme - modalité la plus fréquente (51.9%)
TP          | TP_1           | Temps complet - modalité la plus fréquente (83.2%)
DEPT        | DEPT_51        | Marne - CHOIX UTILISATEUR (département intérieur, 
            |                | coefficients = effet relatif à un dept non-frontalier)
STOCD       | STOCD_10.0     | Propriétaire - modalité la plus fréquente (61.2%)
VOIT        | VOIT_1.0       | 1 voiture - CHOIX UTILISATEUR (référence médiane)
TYPL        | TYPL_1.0       | Maison - modalité la plus fréquente (59.4%)
DNAI        | DNAI_NEGRANDEST| Né Grand Est - modalité la plus fréquente (72.8%)
================================================================================
"""

import pandas as pd
import numpy as np

# ============================================================================
# CHARGEMENT DES DONNÉES
# ============================================================================
print("=" * 80)
print("PRÉPARATION DE LA BASE PROBIT")
print("=" * 80)

# Charger X_final_clean
X = pd.read_csv('/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/ML_READY/FINAL_AFTER_ARBITRAGES/X_final_clean.csv')
print(f"\n✓ Base chargée : {X.shape[0]:,} observations × {X.shape[1]} variables")

# ============================================================================
# DÉFINITION DES CATÉGORIES DE RÉFÉRENCE À SUPPRIMER
# ============================================================================
references_a_supprimer = {
    # Variable catégorielle : colonne de référence à supprimer
    'GS': 'GS_5',              # Employés (26.9%)
    'EMPL': 'EMPL_16',         # CDI/Titulaire (75.8%)
    'INATC': 'INATC_1',        # Français (92.8%)
    'COUPLE': 'COUPLE_1',      # En couple (65.9%)
    'NENFR': 'NENFR_0',        # 0 enfant - CHOIX UTILISATEUR
    'NA5': 'NA5_GU',           # Commerce/Services (45.1%)
    'DIPL': 'DIPL_13',         # CAP/BEP (23.8%)
    'ETUD': 'ETUD_2',          # Non étudiant (94.9%)
    'SANI': 'SANI_2',          # Salle de bain standard (96.1%)
    'SEXE': 'SEXE_1',          # Homme (51.9%)
    'TP': 'TP_1',              # Temps complet (83.2%)
    'DEPT': 'DEPT_51',         # Marne - CHOIX UTILISATEUR
    'STOCD': 'STOCD_10.0',     # Propriétaire (61.2%)
    'VOIT': 'VOIT_1.0',        # 1 voiture - CHOIX UTILISATEUR
    'TYPL': 'TYPL_1.0',        # Maison (59.4%)
    'DNAI': 'DNAI_NEGRANDEST', # Né Grand Est (72.8%)
}

# ============================================================================
# SUPPRESSION DES CATÉGORIES DE RÉFÉRENCE
# ============================================================================
print("\n" + "-" * 80)
print("CATÉGORIES DE RÉFÉRENCE SUPPRIMÉES")
print("-" * 80)

colonnes_a_supprimer = list(references_a_supprimer.values())
colonnes_presentes = [col for col in colonnes_a_supprimer if col in X.columns]
colonnes_absentes = [col for col in colonnes_a_supprimer if col not in X.columns]

if colonnes_absentes:
    print(f"\n⚠ Colonnes non trouvées : {colonnes_absentes}")

# Créer la base probit
bdd_probit = X.drop(columns=colonnes_presentes)

print(f"\n{'Variable':<12} {'Référence supprimée':<20} {'Justification'}")
print("-" * 70)
justifications = {
    'GS_5': 'Employés - plus fréquent (26.9%)',
    'EMPL_16': 'CDI/Titulaire - plus fréquent (75.8%)',
    'INATC_1': 'Français - plus fréquent (92.8%)',
    'COUPLE_1': 'En couple - plus fréquent (65.9%)',
    'NENFR_0': '0 enfant - CHOIX (interprétation parentalité)',
    'NA5_GU': 'Commerce/Services - plus fréquent (45.1%)',
    'DIPL_13': 'CAP/BEP - plus fréquent (23.8%)',
    'ETUD_2': 'Non étudiant - plus fréquent (94.9%)',
    'SANI_2': 'Salle de bain - plus fréquent (96.1%)',
    'SEXE_1': 'Homme - plus fréquent (51.9%)',
    'TP_1': 'Temps complet - plus fréquent (83.2%)',
    'DEPT_51': 'Marne - CHOIX (dept intérieur, non-frontalier)',
    'STOCD_10.0': 'Propriétaire - plus fréquent (61.2%)',
    'VOIT_1.0': '1 voiture - CHOIX (référence médiane)',
    'TYPL_1.0': 'Maison - plus fréquent (59.4%)',
    'DNAI_NEGRANDEST': 'Né Grand Est - plus fréquent (72.8%)',
}

for var, ref in references_a_supprimer.items():
    if ref in colonnes_presentes:
        print(f"{var:<12} {ref:<20} {justifications.get(ref, '')}")

# ============================================================================
# VÉRIFICATION DE LA STRUCTURE FINALE
# ============================================================================
print("\n" + "=" * 80)
print("STRUCTURE DE LA BASE PROBIT")
print("=" * 80)

print(f"\n✓ Dimensions finales : {bdd_probit.shape[0]:,} observations × {bdd_probit.shape[1]} variables")
print(f"  (suppression de {len(colonnes_presentes)} catégories de référence)")

# Compter les modalités par variable conceptuelle
print("\n" + "-" * 80)
print("RÉCAPITULATIF PAR VARIABLE CONCEPTUELLE")
print("-" * 80)
print(f"\n{'Variable':<12} {'Modalités gardées':<40} {'K-1'}")
print("-" * 70)

# Dictionnaire pour regrouper les colonnes par variable
variables_conceptuelles = {
    'AGEREV': ['AGEREV', 'AGEREV_sq'],
    'GS': [c for c in bdd_probit.columns if c.startswith('GS_')],
    'EMPL': [c for c in bdd_probit.columns if c.startswith('EMPL_')],
    'INATC': [c for c in bdd_probit.columns if c.startswith('INATC_')],
    'COUPLE': [c for c in bdd_probit.columns if c.startswith('COUPLE_')],
    'NENFR': [c for c in bdd_probit.columns if c.startswith('NENFR_')],
    'NA5': [c for c in bdd_probit.columns if c.startswith('NA5_')],
    'DIPL': [c for c in bdd_probit.columns if c.startswith('DIPL_')],
    'ETUD': [c for c in bdd_probit.columns if c.startswith('ETUD_')],
    'SANI': [c for c in bdd_probit.columns if c.startswith('SANI_')],
    'SEXE': [c for c in bdd_probit.columns if c.startswith('SEXE_')],
    'TP': [c for c in bdd_probit.columns if c.startswith('TP_')],
    'DEPT': [c for c in bdd_probit.columns if c.startswith('DEPT_')],
    'STOCD': [c for c in bdd_probit.columns if c.startswith('STOCD_')],
    'VOIT': [c for c in bdd_probit.columns if c.startswith('VOIT_')],
    'TYPL': [c for c in bdd_probit.columns if c.startswith('TYPL_')],
    'DNAI': [c for c in bdd_probit.columns if c.startswith('DNAI_')],
}

total_params = 0
for var, cols in variables_conceptuelles.items():
    if cols:
        n_cols = len(cols)
        total_params += n_cols
        cols_str = ', '.join(cols[:3])
        if len(cols) > 3:
            cols_str += f'... (+{len(cols)-3})'
        print(f"{var:<12} {cols_str:<40} {n_cols}")

print("-" * 70)
print(f"{'TOTAL':<12} {'':<40} {total_params} + constante")

# ============================================================================
# LISTE COMPLÈTE DES COLONNES
# ============================================================================
print("\n" + "-" * 80)
print("LISTE COMPLÈTE DES COLONNES (ordre alphabétique)")
print("-" * 80)

for i, col in enumerate(sorted(bdd_probit.columns), 1):
    print(f"  {i:2}. {col}")

# ============================================================================
# SAUVEGARDE
# ============================================================================
import os

out_dir = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/Probit_Ready"
os.makedirs(out_dir, exist_ok=True)

out_file = os.path.join(out_dir, "BDD_PROBIT.csv")   # choisis le nom que tu veux
bdd_probit.to_csv(out_file, index=False)


print("\n" + "=" * 80)
print("SAUVEGARDE")
print("=" * 80)
print(f"\n✓ Base sauvegardée : {output_path}")
print(f"  → {bdd_probit.shape[0]:,} observations")
print(f"  → {bdd_probit.shape[1]} variables explicatives")

# ============================================================================
# APERÇU FINAL
# ============================================================================
print("\n" + "-" * 80)
print("APERÇU (5 premières lignes)")
print("-" * 80)
print(bdd_probit.head())

# ============================================================================
# VÉRIFICATION : RANG DE LA MATRICE
# ============================================================================
print("\n" + "-" * 80)
print("VÉRIFICATION MULTICOLINÉARITÉ")
print("-" * 80)

# Vérifier que la matrice est de plein rang
rank = np.linalg.matrix_rank(bdd_probit.values)
print(f"\n✓ Rang de la matrice X : {rank}")
print(f"✓ Nombre de colonnes   : {bdd_probit.shape[1]}")

if rank == bdd_probit.shape[1]:
    print("\n✅ Matrice de plein rang - pas de multicolinéarité parfaite")
else:
    print(f"\n⚠ ATTENTION : Déficit de rang = {bdd_probit.shape[1] - rank}")
    print("  → Multicolinéarité parfaite détectée")

print("\n" + "=" * 80)
print("PRÉPARATION TERMINÉE")
print("=" * 80)

PRÉPARATION DE LA BASE PROBIT

✓ Base chargée : 494,483 observations × 81 variables

--------------------------------------------------------------------------------
CATÉGORIES DE RÉFÉRENCE SUPPRIMÉES
--------------------------------------------------------------------------------

Variable     Référence supprimée  Justification
----------------------------------------------------------------------
GS           GS_5                 Employés - plus fréquent (26.9%)
EMPL         EMPL_16              CDI/Titulaire - plus fréquent (75.8%)
INATC        INATC_1              Français - plus fréquent (92.8%)
COUPLE       COUPLE_1             En couple - plus fréquent (65.9%)
NENFR        NENFR_0              0 enfant - CHOIX (interprétation parentalité)
NA5          NA5_GU               Commerce/Services - plus fréquent (45.1%)
DIPL         DIPL_13              CAP/BEP - plus fréquent (23.8%)
ETUD         ETUD_2               Non étudiant - plus fréquent (94.9%)
SANI         SANI_2            

La BDD est prêt pour le lancement de l'analyse statistique...

# Estimation économétrique en modèle probit binaire : Les déterminants du travail transfrontalier

In [8]:
"""
================================================================================
MODÈLE 1 : PROBIT BINAIRE - DÉTERMINANTS DU TRAVAIL TRANSFRONTALIER
================================================================================
"""

import pandas as pd
import numpy as np
import statsmodels.api as sm

# ============================================================================
# CHARGEMENT
# ============================================================================
path = '/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/Probit_Ready/'
X = pd.read_csv(path + 'BDD_PROBIT.csv')
y = pd.read_csv(path + 'y_final.csv')['TRANSFRONTALIER']

# Ajouter constante
X_const = sm.add_constant(X)

print(f"Observations : {len(y):,}")
print(f"Transfrontaliers : {y.sum():,} ({y.mean()*100:.2f}%)")
print(f"Variables : {X_const.shape[1]} (dont constante)")

# ============================================================================
# ESTIMATION PROBIT
# ============================================================================
print("\nEstimation en cours...")

probit = sm.Probit(y, X_const)
result = probit.fit(disp=1, maxiter=100)

# ============================================================================
# RÉSULTATS
# ============================================================================
print(result.summary())

# ============================================================================
# EFFETS MARGINAUX (à la moyenne)
# ============================================================================
marginal_effects = result.get_margeff(at='mean')
print("\n" + "="*60)
print("EFFETS MARGINAUX (à la moyenne)")
print("="*60)
print(marginal_effects.summary())

# ============================================================================
# SAUVEGARDE
# ============================================================================
with open(path + 'probit_model1_summary.txt', 'w') as f:
    f.write(result.summary().as_text())
    f.write("\n\n" + "="*60 + "\n")
    f.write("EFFETS MARGINAUX\n")
    f.write("="*60 + "\n")
    f.write(marginal_effects.summary().as_text())

print(f"\n✓ Résultats sauvegardés : probit_model1_summary.txt")

Observations : 494,483
Transfrontaliers : 44,264 (8.95%)
Variables : 66 (dont constante)

Estimation en cours...
Optimization terminated successfully.
         Current function value: 0.229292
         Iterations 10
                          Probit Regression Results                           
Dep. Variable:        TRANSFRONTALIER   No. Observations:               494483
Model:                         Probit   Df Residuals:                   494417
Method:                           MLE   Df Model:                           65
Date:                Wed, 07 Jan 2026   Pseudo R-squ.:                  0.2393
Time:                        12:27:56   Log-Likelihood:            -1.1338e+05
converged:                       True   LL-Null:                   -1.4905e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                      coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------

## Résultats préliminaires du modèle probit

### Convention de lecture

Les résultats sont présentés sous deux formes complémentaires :

1. **Coefficient probit** : paramètre estimé du modèle, dont le signe indique la direction de l'effet mais dont l'amplitude n'est pas directement interprétable.

2. **Effet marginal (pp)** : variation de la probabilité d'être transfrontalier en points de pourcentage, calculé à la moyenne des variables. *Exemple : +5 pp signifie que la probabilité passe de 8,95% à 13,95%.*

La catégorie de référence pour les variables départementales est la Marne (DEPT_51), département non frontalier.

### Classification des variables selon le risque d'endogénéité

| Statut | Interprétation | Variables concernées |
|--------|----------------|----------------------|
| **Exogène** | Relation causale plausible | Département, âge, sexe, nationalité, lieu de naissance, diplôme |
| **Probablement exogène** | Causalité vraisemblable | Nombre d'enfants, secteur d'activité, catégorie socioprofessionnelle |
| **Ambigu** | Causalité incertaine | Type de contrat, temps de travail |
| **Potentiellement endogène** | Causalité inverse possible | Statut d'occupation, nombre de voitures, type de logement |

Cette classification guide l'interprétation : seules les variables exogènes autorisent une lecture causale. Les variables potentiellement endogènes sont incluses comme contrôles mais leurs coefficients ne doivent pas être interprétés causalement.

---

### Qualité globale de l'estimation

Le modèle converge en 10 itérations sur 494 483 observations (44 264 transfrontaliers, soit 8,95%). Le pseudo-R² de McFadden s'établit à **0,239**, valeur satisfaisante pour un modèle de choix discret sur données individuelles. Le test du ratio de vraisemblance rejette la nullité jointe des coefficients (χ² = 71 340, p < 0,001). Sur 65 variables, 56 sont significatives au seuil de 1%.

---

### Patterns observés

Les coefficients estimés révèlent plusieurs régularités cohérentes avec la littérature et les attentes théoriques :

**Gradient géographique** : Les départements frontaliers affichent les effets les plus forts du modèle. La Moselle, le Haut-Rhin et la Meurthe-et-Moselle présentent des effets marginaux de +12 à +15 pp, reflétant la proximité des bassins d'emploi luxembourgeois, suisse et allemand.

**Profil démographique** : L'âge suit une forme quadratique suggérant un pic vers 40 ans. Les femmes présentent une probabilité inférieure (coefficient négatif significatif). La présence d'enfants exerce un effet négatif croissant avec leur nombre.

**Avantage des réseaux transnationaux** : La nationalité étrangère et la naissance à l'étranger sont positivement associées au travail frontalier, possiblement via les compétences linguistiques et les réseaux professionnels.

**Structure professionnelle** : Les ouvriers en CDI sont favorisés, tandis que les indépendants, le secteur public et l'agriculture sont fortement exclus. Le travail frontalier apparaît comme un phénomène essentiellement salarié.

**Capital humain** : La relation avec le diplôme est non monotone, avec des probabilités élevées aux deux extrémités (CAP-BEP d'un côté, master-doctorat de l'autre) et plus faibles pour les diplômes intermédiaires.

**Associations avec le logement et la motorisation** : Des corrélations significatives sont observées (propriétaires et multi-motorisés plus souvent transfrontaliers), mais le sens de la causalité reste indéterminé à ce stade.

---

### Pourquoi l'inférence reste provisoire

Ces régularités statistiques ne permettent pas encore de conclusions causales. Plusieurs questions méthodologiques doivent être tranchées par des tests de robustesse :

- **Spécification fonctionnelle** : la forme quadratique de l'âge est-elle optimale ?
- **Qualité de l'ajustement** : les tests classiques (Hosmer-Lemeshow, Link Test) valident-ils le modèle ?
- **Équivalence Probit/Logit** : les conclusions sont-elles robustes au choix du modèle ?
- **Sensibilité à l'endogénéité** : les coefficients clés sont-ils stables après exclusion des variables potentiellement endogènes ?
- **Hétérogénéité** : la structure est-elle homogène par sexe, âge, département ?
- **Observations atypiques** : existe-t-il des résidus extrêmes influençant les résultats ?

L'interprétation économique détaillée et les conclusions causales seront présentées après validation de ces diagnostics.

# Tests de Robustesse – Modèle Probit Transfrontalier

## Tests réalisés

| #   | Test                             | Objectif                                    | Critère de validation                 |
|-----|----------------------------------|---------------------------------------------|---------------------------------------|
| 1   | Probit de base                   | Vérifier convergence et qualité globale     | Convergence, Pseudo R², LR test p<0.05|
| 2   | Link Test (Pregibon)             | Détecter erreur de spécification            | p(_hatsq) > 0.05                      |
| 3   | Hosmer-Lemeshow                  | Tester l'ajustement du modèle               | p > 0.05                              |
| 4   | Probit vs Logit                  | Vérifier robustesse au choix du lien        | Corr. effets marginaux > 0.99         |
| 5   | Exclusion variables endogènes    | Sensibilité à VOIT, STOCD, TYPL, SANI       | Variation coefficients clés < 20%     |
| 6   | Sous-échantillons                | Stabilité selon groupes                     | Cohérence des Pseudo R²               |
| 7   | Sensibilité                      | Influence des observations extrêmes         | Stabilité après exclusion             |
| 8   | Forme fonctionnelle âge          | Valider la spécification quadratique        | Test LR quad vs lin, AIC/BIC          |

## Sous-échantillons analysés (Test 6)

| Dimension   | Groupes comparés                      |
|-------------|---------------------------------------|
| Géographie  | Départements frontaliers vs intérieurs|
| Genre       | Hommes vs Femmes                      |
| Âge         | < 35 ans vs ≥ 35 ans                  |

## Output

→ `rapport_robustesse.txt` dans `/Probit_Ready/Robustesse/`

In [1]:
"""
================================================================================
TESTS DE ROBUSTESSE - MODÈLE PROBIT TRANSFRONTALIER
================================================================================
Version adaptée pour BDD_PROBIT.csv (base déjà préparée)

Utilise directement :
- X : BDD_PROBIT.csv (variables explicatives, catégories de référence déjà supprimées)
- Y : y_final.csv (variable dépendante)

Catégories de référence (rappel) :
- DEPT_51 (Marne), SEXE_1 (Homme), GS_5 (Employés), EMPL_16 (CDI)
- NENFR_0 (0 enfant), NA5_GU (Commerce/Services), DIPL_13 (CAP/BEP)
- COUPLE_1 (En couple), TP_1 (Temps complet), etc.
================================================================================
"""

import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
from scipy.stats import chi2
import warnings
import os
from datetime import datetime

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION - CHEMINS DES FICHIERS
# ============================================================================
# Adapter ces chemins si nécessaire
X_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/Probit_Ready/BDD_PROBIT.csv"
Y_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/Probit_Ready/y_final.csv"
OUTPUT_DIR = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/Probit_Ready/Robustesse"


def load_data(x_path, y_path):
    """
    Charge les données X et Y déjà préparées.
    """
    print("=" * 80)
    print("CHARGEMENT DES DONNÉES")
    print("=" * 80)
    
    # Charger X
    X = pd.read_csv(x_path)
    print(f"\n✓ X chargé : {X.shape[0]:,} obs × {X.shape[1]} variables")
    
    # Charger Y
    Y_df = pd.read_csv(y_path)
    print(f"✓ Y chargé : {Y_df.shape[0]:,} observations")
    
    # Identifier la colonne Y (peut être 'Y', 'y', 'ILT_7', 'TRANSFRONTALIER', etc.)
    y_col = Y_df.columns[0]  # Prendre la première colonne
    Y = Y_df[y_col].values
    print(f"  → Colonne utilisée : '{y_col}'")
    
    # Statistiques Y
    n_transfront = Y.sum()
    pct_transfront = 100 * n_transfront / len(Y)
    print(f"\n✓ Variable dépendante :")
    print(f"  → Transfrontaliers (Y=1) : {n_transfront:,} ({pct_transfront:.2f}%)")
    print(f"  → Non-transfrontaliers   : {len(Y) - n_transfront:,} ({100-pct_transfront:.2f}%)")
    
    # Vérifier cohérence
    if len(X) != len(Y):
        raise ValueError(f"Incohérence : X a {len(X)} obs, Y a {len(Y)} obs")
    
    # Ajouter constante
    X_with_const = sm.add_constant(X, has_constant='add')
    print(f"\n✓ Constante ajoutée : {X_with_const.shape[1]} variables (dont intercept)")
    
    return X, X_with_const, Y


def test_1_baseline_probit(X, Y, report_lines):
    """
    Test 1 : Estimation du modèle probit de base
    """
    print("\n" + "=" * 80)
    print("TEST 1 : MODÈLE PROBIT DE BASE")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 1 : MODÈLE PROBIT DE BASE")
    report_lines.append("=" * 80)
    
    try:
        model = sm.Probit(Y, X)
        result = model.fit(disp=0, maxiter=100)
        
        print(f"\n✓ Convergence : {'Oui' if result.mle_retvals['converged'] else 'Non'}")
        print(f"✓ Itérations  : {result.mle_retvals.get('iterations', 'N/A')}")
        print(f"✓ Log-L       : {result.llf:,.2f}")
        print(f"✓ Pseudo R²   : {result.prsquared:.4f}")
        print(f"✓ AIC         : {result.aic:,.2f}")
        print(f"✓ BIC         : {result.bic:,.2f}")
        
        # Test LR global
        llr_pvalue = result.llr_pvalue
        print(f"✓ LR test p   : {llr_pvalue:.2e}")
        
        report_lines.append(f"\nConvergence : {'Oui' if result.mle_retvals['converged'] else 'Non'}")
        report_lines.append(f"Itérations  : {result.mle_retvals.get('iterations', 'N/A')}")
        report_lines.append(f"Log-L       : {result.llf:,.2f}")
        report_lines.append(f"Pseudo R²   : {result.prsquared:.4f}")
        report_lines.append(f"AIC         : {result.aic:,.2f}")
        report_lines.append(f"BIC         : {result.bic:,.2f}")
        report_lines.append(f"LR test p   : {llr_pvalue:.2e}")
        
        # Coefficients clés
        print("\n--- Coefficients clés ---")
        report_lines.append("\n--- Coefficients clés ---")
        
        key_vars = ['DEPT_57', 'DEPT_68', 'DEPT_54', 'DEPT_67', 'SEXE_2', 'AGEREV', 'AGEREV_sq']
        for var in key_vars:
            if var in result.params.index:
                coef = result.params[var]
                se = result.bse[var]
                pval = result.pvalues[var]
                signif = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
                print(f"  {var:<12} : {coef:>8.4f} (SE={se:.4f}) {signif}")
                report_lines.append(f"  {var:<12} : {coef:>8.4f} (SE={se:.4f}) {signif}")
        
        report_lines.append("\n✅ TEST 1 RÉUSSI")
        return result
        
    except Exception as e:
        print(f"\n❌ Erreur : {e}")
        report_lines.append(f"\n❌ ERREUR : {e}")
        return None


def test_2_link_test(X, Y, baseline_result, report_lines):
    """
    Test 2 : Link Test de Pregibon
    H0 : Spécification correcte (coefficient de _hatsq non significatif)
    """
    print("\n" + "=" * 80)
    print("TEST 2 : LINK TEST (PREGIBON)")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 2 : LINK TEST (PREGIBON)")
    report_lines.append("=" * 80)
    
    try:
        # Prédictions linéaires
        y_hat = baseline_result.predict()
        xb = stats.norm.ppf(np.clip(y_hat, 1e-10, 1-1e-10))  # Inverse de Phi
        xb_sq = xb ** 2
        
        # Régression auxiliaire
        X_link = pd.DataFrame({
            'const': 1,
            '_hat': xb,
            '_hatsq': xb_sq
        })
        
        model_link = sm.Probit(Y, X_link)
        result_link = model_link.fit(disp=0)
        
        coef_hatsq = result_link.params['_hatsq']
        pval_hatsq = result_link.pvalues['_hatsq']
        
        print(f"\n_hat   coef : {result_link.params['_hat']:.4f} (p={result_link.pvalues['_hat']:.4f})")
        print(f"_hatsq coef : {coef_hatsq:.4f} (p={pval_hatsq:.4f})")
        
        report_lines.append(f"\n_hat   coef : {result_link.params['_hat']:.4f} (p={result_link.pvalues['_hat']:.4f})")
        report_lines.append(f"_hatsq coef : {coef_hatsq:.4f} (p={pval_hatsq:.4f})")
        
        if pval_hatsq > 0.05:
            print("\n✅ H0 non rejetée : Spécification correcte")
            report_lines.append("\n✅ RÉUSSI : Spécification correcte (p > 0.05)")
        else:
            print("\n⚠️ H0 rejetée : Possible erreur de spécification")
            report_lines.append("\n⚠️ ATTENTION : Possible erreur de spécification (p < 0.05)")
            
        return pval_hatsq
        
    except Exception as e:
        print(f"\n❌ Erreur : {e}")
        report_lines.append(f"\n❌ ERREUR : {e}")
        return None


def test_3_hosmer_lemeshow(Y, baseline_result, report_lines, n_groups=10):
    """
    Test 3 : Hosmer-Lemeshow Goodness of Fit
    """
    print("\n" + "=" * 80)
    print("TEST 3 : HOSMER-LEMESHOW")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 3 : HOSMER-LEMESHOW")
    report_lines.append("=" * 80)
    
    try:
        y_pred = baseline_result.predict()
        
        # Créer déciles
        df_hl = pd.DataFrame({'y': Y, 'pred': y_pred})
        df_hl['decile'] = pd.qcut(df_hl['pred'], n_groups, labels=False, duplicates='drop')
        
        # Calculer observé vs attendu par groupe
        grouped = df_hl.groupby('decile').agg({
            'y': ['sum', 'count'],
            'pred': 'sum'
        })
        grouped.columns = ['obs_1', 'n', 'exp_1']
        grouped['obs_0'] = grouped['n'] - grouped['obs_1']
        grouped['exp_0'] = grouped['n'] - grouped['exp_1']
        
        # Statistique HL
        hl_stat = 0
        for _, row in grouped.iterrows():
            if row['exp_1'] > 0:
                hl_stat += (row['obs_1'] - row['exp_1'])**2 / row['exp_1']
            if row['exp_0'] > 0:
                hl_stat += (row['obs_0'] - row['exp_0'])**2 / row['exp_0']
        
        df_hl_test = len(grouped) - 2
        p_value = 1 - chi2.cdf(hl_stat, df_hl_test)
        
        print(f"\nStatistique HL : {hl_stat:.2f}")
        print(f"Degrés liberté : {df_hl_test}")
        print(f"P-value        : {p_value:.4f}")
        
        report_lines.append(f"\nStatistique HL : {hl_stat:.2f}")
        report_lines.append(f"Degrés liberté : {df_hl_test}")
        report_lines.append(f"P-value        : {p_value:.4f}")
        
        if p_value > 0.05:
            print("\n✅ Bon ajustement (p > 0.05)")
            report_lines.append("\n✅ RÉUSSI : Bon ajustement")
        else:
            print("\n⚠️ Ajustement insuffisant (p < 0.05)")
            report_lines.append("\n⚠️ ATTENTION : Ajustement insuffisant")
            
        return hl_stat, p_value
        
    except Exception as e:
        print(f"\n❌ Erreur : {e}")
        report_lines.append(f"\n❌ ERREUR : {e}")
        return None, None


def test_4_probit_vs_logit(X, Y, report_lines):
    """
    Test 4 : Comparaison Probit vs Logit
    """
    print("\n" + "=" * 80)
    print("TEST 4 : PROBIT VS LOGIT")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 4 : PROBIT VS LOGIT")
    report_lines.append("=" * 80)
    
    try:
        # Probit
        model_probit = sm.Probit(Y, X)
        result_probit = model_probit.fit(disp=0)
        
        # Logit
        model_logit = sm.Logit(Y, X)
        result_logit = model_logit.fit(disp=0)
        
        print(f"\n{'Critère':<15} {'Probit':>12} {'Logit':>12}")
        print("-" * 45)
        print(f"{'Log-L':<15} {result_probit.llf:>12,.2f} {result_logit.llf:>12,.2f}")
        print(f"{'Pseudo R²':<15} {result_probit.prsquared:>12.4f} {result_logit.prsquared:>12.4f}")
        print(f"{'AIC':<15} {result_probit.aic:>12,.2f} {result_logit.aic:>12,.2f}")
        print(f"{'BIC':<15} {result_probit.bic:>12,.2f} {result_logit.bic:>12,.2f}")
        
        report_lines.append(f"\n{'Critère':<15} {'Probit':>12} {'Logit':>12}")
        report_lines.append("-" * 45)
        report_lines.append(f"{'Log-L':<15} {result_probit.llf:>12,.2f} {result_logit.llf:>12,.2f}")
        report_lines.append(f"{'Pseudo R²':<15} {result_probit.prsquared:>12.4f} {result_logit.prsquared:>12.4f}")
        report_lines.append(f"{'AIC':<15} {result_probit.aic:>12,.2f} {result_logit.aic:>12,.2f}")
        report_lines.append(f"{'BIC':<15} {result_probit.bic:>12,.2f} {result_logit.bic:>12,.2f}")
        
        # Ratio des coefficients (théoriquement ~1.6)
        common_vars = [v for v in result_probit.params.index if v in result_logit.params.index and v != 'const']
        if common_vars:
            ratios = result_logit.params[common_vars] / result_probit.params[common_vars]
            mean_ratio = ratios.mean()
            print(f"\nRatio moyen Logit/Probit : {mean_ratio:.3f} (théorique ~1.6)")
            report_lines.append(f"\nRatio moyen Logit/Probit : {mean_ratio:.3f} (théorique ~1.6)")
        
        # Corrélation des effets marginaux
        mfx_probit = result_probit.get_margeff(at='mean').margeff
        mfx_logit = result_logit.get_margeff(at='mean').margeff
        corr_mfx = np.corrcoef(mfx_probit, mfx_logit)[0, 1]
        
        print(f"Corrélation effets marginaux : {corr_mfx:.4f}")
        report_lines.append(f"Corrélation effets marginaux : {corr_mfx:.4f}")
        
        if corr_mfx > 0.99:
            print("\n✅ Probit et Logit quasi-équivalents")
            report_lines.append("\n✅ RÉUSSI : Probit et Logit quasi-équivalents")
        else:
            print("\n⚠️ Différence notable entre Probit et Logit")
            report_lines.append("\n⚠️ ATTENTION : Différence notable")
            
        return result_probit, result_logit, corr_mfx
        
    except Exception as e:
        print(f"\n❌ Erreur : {e}")
        report_lines.append(f"\n❌ ERREUR : {e}")
        return None, None, None


def test_5_endogeneity_robustness(X, Y, baseline_result, report_lines):
    """
    Test 5 : Robustesse à l'exclusion des variables potentiellement endogènes
    """
    print("\n" + "=" * 80)
    print("TEST 5 : ROBUSTESSE ENDOGÉNÉITÉ")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 5 : ROBUSTESSE ENDOGÉNÉITÉ")
    report_lines.append("=" * 80)
    
    # Variables potentiellement endogènes à exclure
    vars_endogenes = ['VOIT', 'STOCD', 'TYPL', 'SANI']
    
    try:
        # Identifier les colonnes à exclure
        cols_to_drop = []
        for var in vars_endogenes:
            cols_to_drop.extend([c for c in X.columns if c.startswith(f'{var}_') or c == var])
        
        cols_to_drop = [c for c in cols_to_drop if c in X.columns]
        
        if not cols_to_drop:
            print("\n⚠️ Aucune variable endogène trouvée dans X")
            report_lines.append("\n⚠️ Aucune variable endogène trouvée")
            return None
        
        print(f"\nVariables exclues : {len(cols_to_drop)}")
        for c in cols_to_drop[:10]:
            print(f"  - {c}")
        if len(cols_to_drop) > 10:
            print(f"  ... et {len(cols_to_drop)-10} autres")
        
        # Modèle réduit
        X_reduced = X.drop(columns=cols_to_drop)
        X_reduced_const = sm.add_constant(X_reduced)
        
        model_reduced = sm.Probit(Y, X_reduced_const)
        result_reduced = model_reduced.fit(disp=0)
        
        print(f"\n{'Modèle':<12} {'Log-L':>12} {'Pseudo R²':>12} {'Nb vars':>10}")
        print("-" * 50)
        print(f"{'Complet':<12} {baseline_result.llf:>12,.2f} {baseline_result.prsquared:>12.4f} {len(baseline_result.params):>10}")
        print(f"{'Réduit':<12} {result_reduced.llf:>12,.2f} {result_reduced.prsquared:>12.4f} {len(result_reduced.params):>10}")
        
        report_lines.append(f"\nVariables exclues : {cols_to_drop}")
        report_lines.append(f"\n{'Modèle':<12} {'Log-L':>12} {'Pseudo R²':>12}")
        report_lines.append("-" * 50)
        report_lines.append(f"{'Complet':<12} {baseline_result.llf:>12,.2f} {baseline_result.prsquared:>12.4f}")
        report_lines.append(f"{'Réduit':<12} {result_reduced.llf:>12,.2f} {result_reduced.prsquared:>12.4f}")
        
        # Stabilité des coefficients clés
        print("\n--- Stabilité des coefficients clés ---")
        report_lines.append("\n--- Stabilité des coefficients clés ---")
        
        key_vars = ['DEPT_57', 'DEPT_68', 'SEXE_2', 'AGEREV']
        stable = True
        
        for var in key_vars:
            if var in baseline_result.params.index and var in result_reduced.params.index:
                coef_full = baseline_result.params[var]
                coef_red = result_reduced.params[var]
                pct_change = 100 * (coef_red - coef_full) / abs(coef_full) if coef_full != 0 else 0
                
                status = "✓" if abs(pct_change) < 20 else "⚠️"
                if abs(pct_change) >= 20:
                    stable = False
                    
                print(f"  {var:<12} : {coef_full:>8.4f} → {coef_red:>8.4f} ({pct_change:+.1f}%) {status}")
                report_lines.append(f"  {var:<12} : {coef_full:>8.4f} → {coef_red:>8.4f} ({pct_change:+.1f}%) {status}")
        
        if stable:
            print("\n✅ Coefficients stables après exclusion")
            report_lines.append("\n✅ RÉUSSI : Coefficients stables")
        else:
            print("\n⚠️ Certains coefficients varient de plus de 20%")
            report_lines.append("\n⚠️ ATTENTION : Coefficients instables")
            
        return result_reduced
        
    except Exception as e:
        print(f"\n❌ Erreur : {e}")
        report_lines.append(f"\n❌ ERREUR : {e}")
        return None


def test_6_subsamples(X, Y, report_lines):
    """
    Test 6 : Analyse par sous-échantillons
    """
    print("\n" + "=" * 80)
    print("TEST 6 : SOUS-ÉCHANTILLONS")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 6 : SOUS-ÉCHANTILLONS")
    report_lines.append("=" * 80)
    
    results_subsamples = {}
    
    # 1. Par département (frontalier vs intérieur)
    print("\n--- Départements frontaliers vs intérieurs ---")
    report_lines.append("\n--- Départements frontaliers vs intérieurs ---")
    
    # Identifier les départements frontaliers
    dept_front_cols = ['DEPT_57', 'DEPT_68', 'DEPT_54', 'DEPT_67', 'DEPT_55', 'DEPT_88', 'DEPT_90']
    dept_front_cols = [c for c in dept_front_cols if c in X.columns]
    
    if dept_front_cols:
        mask_front = X[dept_front_cols].sum(axis=1) > 0
        
        for name, mask in [('Frontaliers', mask_front), ('Intérieurs', ~mask_front)]:
            X_sub = X.loc[mask]
            Y_sub = Y[mask.values]
            
            if len(Y_sub) > 100 and Y_sub.sum() > 10:
                X_sub_const = sm.add_constant(X_sub)
                try:
                    model = sm.Probit(Y_sub, X_sub_const)
                    result = model.fit(disp=0, maxiter=50)
                    prevalence = 100 * Y_sub.mean()
                    print(f"  {name:<12} : n={len(Y_sub):,}, Y=1: {prevalence:.1f}%, Pseudo R²={result.prsquared:.4f}")
                    report_lines.append(f"  {name:<12} : n={len(Y_sub):,}, Y=1: {prevalence:.1f}%, Pseudo R²={result.prsquared:.4f}")
                    results_subsamples[name] = result
                except:
                    print(f"  {name:<12} : Estimation échouée")
    
    # 2. Par sexe
    print("\n--- Par sexe ---")
    report_lines.append("\n--- Par sexe ---")
    
    if 'SEXE_2' in X.columns:
        for name, val in [('Hommes', 0), ('Femmes', 1)]:
            mask = X['SEXE_2'] == val
            X_sub = X.loc[mask].drop(columns=['SEXE_2'])
            Y_sub = Y[mask.values]
            
            if len(Y_sub) > 100 and Y_sub.sum() > 10:
                X_sub_const = sm.add_constant(X_sub)
                try:
                    model = sm.Probit(Y_sub, X_sub_const)
                    result = model.fit(disp=0, maxiter=50)
                    prevalence = 100 * Y_sub.mean()
                    print(f"  {name:<12} : n={len(Y_sub):,}, Y=1: {prevalence:.1f}%, Pseudo R²={result.prsquared:.4f}")
                    report_lines.append(f"  {name:<12} : n={len(Y_sub):,}, Y=1: {prevalence:.1f}%, Pseudo R²={result.prsquared:.4f}")
                    results_subsamples[name] = result
                except:
                    print(f"  {name:<12} : Estimation échouée")
    
    # 3. Par âge
    print("\n--- Par tranche d'âge ---")
    report_lines.append("\n--- Par tranche d'âge ---")
    
    if 'AGEREV' in X.columns:
        for name, condition in [('<35 ans', X['AGEREV'] < 35), ('≥35 ans', X['AGEREV'] >= 35)]:
            mask = condition
            X_sub = X.loc[mask]
            Y_sub = Y[mask.values]
            
            if len(Y_sub) > 100 and Y_sub.sum() > 10:
                X_sub_const = sm.add_constant(X_sub)
                try:
                    model = sm.Probit(Y_sub, X_sub_const)
                    result = model.fit(disp=0, maxiter=50)
                    prevalence = 100 * Y_sub.mean()
                    print(f"  {name:<12} : n={len(Y_sub):,}, Y=1: {prevalence:.1f}%, Pseudo R²={result.prsquared:.4f}")
                    report_lines.append(f"  {name:<12} : n={len(Y_sub):,}, Y=1: {prevalence:.1f}%, Pseudo R²={result.prsquared:.4f}")
                    results_subsamples[name] = result
                except:
                    print(f"  {name:<12} : Estimation échouée")
    
    report_lines.append("\n✅ Analyse par sous-échantillons terminée")
    return results_subsamples


def test_7_sensitivity(X, Y, baseline_result, report_lines):
    """
    Test 7 : Analyse de sensibilité
    """
    print("\n" + "=" * 80)
    print("TEST 7 : SENSIBILITÉ")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 7 : SENSIBILITÉ")
    report_lines.append("=" * 80)
    
    # 1. Exclusion des résidus extrêmes
    print("\n--- Exclusion résidus extrêmes ---")
    report_lines.append("\n--- Exclusion résidus extrêmes ---")
    
    try:
        y_pred = baseline_result.predict()
        residuals = Y - y_pred
        std_res = (residuals - residuals.mean()) / residuals.std()
        
        mask_normal = np.abs(std_res) <= 3
        n_excluded = (~mask_normal).sum()
        
        print(f"  Observations exclues (|résidu| > 3σ) : {n_excluded} ({100*n_excluded/len(Y):.2f}%)")
        report_lines.append(f"  Observations exclues : {n_excluded} ({100*n_excluded/len(Y):.2f}%)")
        
        X_clean = X.loc[mask_normal]
        Y_clean = Y[mask_normal]
        X_clean_const = sm.add_constant(X_clean)
        
        model_clean = sm.Probit(Y_clean, X_clean_const)
        result_clean = model_clean.fit(disp=0)
        
        print(f"  Pseudo R² (sans extrêmes) : {result_clean.prsquared:.4f}")
        report_lines.append(f"  Pseudo R² (sans extrêmes) : {result_clean.prsquared:.4f}")
        
    except Exception as e:
        print(f"  ❌ Erreur : {e}")
    
    # 2. Jackknife par département
    print("\n--- Jackknife par département ---")
    report_lines.append("\n--- Jackknife par département ---")
    
    dept_cols = [c for c in X.columns if c.startswith('DEPT_')]
    
    key_var = 'AGEREV' if 'AGEREV' in X.columns else (dept_cols[0] if dept_cols else None)
    
    if key_var and dept_cols:
        coefs_jack = []
        
        for dept in dept_cols[:5]:  # Limiter à 5 pour la vitesse
            mask = X[dept] == 0  # Exclure ce département
            X_jack = X.loc[mask]
            Y_jack = Y[mask.values]
            
            if len(Y_jack) > 100:
                try:
                    X_jack_const = sm.add_constant(X_jack)
                    model = sm.Probit(Y_jack, X_jack_const)
                    result = model.fit(disp=0, maxiter=30)
                    
                    if key_var in result.params.index:
                        coefs_jack.append(result.params[key_var])
                        print(f"  Sans {dept:<10} : {key_var} = {result.params[key_var]:.4f}")
                        report_lines.append(f"  Sans {dept:<10} : {key_var} = {result.params[key_var]:.4f}")
                except:
                    pass
        
        if coefs_jack:
            std_jack = np.std(coefs_jack)
            print(f"\n  Écart-type Jackknife ({key_var}) : {std_jack:.4f}")
            report_lines.append(f"\n  Écart-type Jackknife ({key_var}) : {std_jack:.4f}")
    
    report_lines.append("\n✅ Analyse de sensibilité terminée")
    return True


def test_8_functional_form_age(X, Y, report_lines):
    """
    Test 8 : Formes fonctionnelles pour l'âge
    """
    print("\n" + "=" * 80)
    print("TEST 8 : FORME FONCTIONNELLE ÂGE")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 8 : FORME FONCTIONNELLE ÂGE")
    report_lines.append("=" * 80)
    
    if 'AGEREV' not in X.columns:
        print("\n⚠️ Variable AGEREV non trouvée")
        report_lines.append("\n⚠️ Variable AGEREV non trouvée")
        return None
    
    results_age = {}
    
    # Préparer les variables autres que l'âge
    age_cols = [c for c in X.columns if 'AGE' in c.upper()]
    X_base = X.drop(columns=age_cols, errors='ignore')
    
    try:
        # 1. Linéaire
        X_lin = X_base.copy()
        X_lin['AGE'] = X['AGEREV']
        X_lin_const = sm.add_constant(X_lin)
        model_lin = sm.Probit(Y, X_lin_const)
        result_lin = model_lin.fit(disp=0)
        results_age['Linéaire'] = result_lin
        
        # 2. Quadratique
        X_quad = X_base.copy()
        X_quad['AGE'] = X['AGEREV']
        X_quad['AGE_sq'] = X['AGEREV'] ** 2
        X_quad_const = sm.add_constant(X_quad)
        model_quad = sm.Probit(Y, X_quad_const)
        result_quad = model_quad.fit(disp=0)
        results_age['Quadratique'] = result_quad
        
        # 3. Cubique
        X_cub = X_base.copy()
        X_cub['AGE'] = X['AGEREV']
        X_cub['AGE_sq'] = X['AGEREV'] ** 2
        X_cub['AGE_cub'] = X['AGEREV'] ** 3
        X_cub_const = sm.add_constant(X_cub)
        model_cub = sm.Probit(Y, X_cub_const)
        result_cub = model_cub.fit(disp=0)
        results_age['Cubique'] = result_cub
        
        # 4. Spline (tranches d'âge)
        X_spline = X_base.copy()
        X_spline['AGE_25_34'] = ((X['AGEREV'] >= 25) & (X['AGEREV'] < 35)).astype(int)
        X_spline['AGE_35_44'] = ((X['AGEREV'] >= 35) & (X['AGEREV'] < 45)).astype(int)
        X_spline['AGE_45_54'] = ((X['AGEREV'] >= 45) & (X['AGEREV'] < 55)).astype(int)
        X_spline['AGE_55plus'] = (X['AGEREV'] >= 55).astype(int)
        X_spline_const = sm.add_constant(X_spline)
        model_spline = sm.Probit(Y, X_spline_const)
        result_spline = model_spline.fit(disp=0)
        results_age['Spline'] = result_spline
        
        # Comparaison
        print(f"\n{'Forme':<15} {'Log-L':>12} {'AIC':>12} {'BIC':>12}")
        print("-" * 55)
        report_lines.append(f"\n{'Forme':<15} {'Log-L':>12} {'AIC':>12} {'BIC':>12}")
        report_lines.append("-" * 55)
        
        for name, res in results_age.items():
            print(f"{name:<15} {res.llf:>12,.2f} {res.aic:>12,.2f} {res.bic:>12,.2f}")
            report_lines.append(f"{name:<15} {res.llf:>12,.2f} {res.aic:>12,.2f} {res.bic:>12,.2f}")
        
        # Meilleur selon BIC
        best = min(results_age.items(), key=lambda x: x[1].bic)
        print(f"\n✓ Meilleure forme (BIC) : {best[0]}")
        report_lines.append(f"\n✓ Meilleure forme (BIC) : {best[0]}")
        
        # Test LR quadratique vs linéaire
        lr_stat = 2 * (result_quad.llf - result_lin.llf)
        lr_pval = 1 - chi2.cdf(lr_stat, 1)
        print(f"\nTest LR (Quad vs Lin) : χ²={lr_stat:.2f}, p={lr_pval:.4f}")
        report_lines.append(f"\nTest LR (Quad vs Lin) : χ²={lr_stat:.2f}, p={lr_pval:.4f}")
        
        if lr_pval < 0.05:
            print("✓ Effet quadratique significatif")
            report_lines.append("✓ Effet quadratique significatif")
        
        return results_age
        
    except Exception as e:
        print(f"\n❌ Erreur : {e}")
        report_lines.append(f"\n❌ ERREUR : {e}")
        return None


def generate_report(report_lines, output_dir):
    """
    Génère le rapport final
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Ajouter en-tête
    header = [
        "=" * 80,
        "RAPPORT DE ROBUSTESSE - MODÈLE PROBIT TRANSFRONTALIER",
        f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        "=" * 80,
        ""
    ]
    
    full_report = header + report_lines
    
    # Sauvegarder
    report_path = os.path.join(output_dir, "rapport_robustesse.txt")
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(full_report))
    
    print(f"\n✓ Rapport sauvegardé : {report_path}")
    return report_path


def run_all_tests(x_path=X_PATH, y_path=Y_PATH, output_dir=OUTPUT_DIR):
    """
    Exécute tous les tests de robustesse
    """
    report_lines = []
    
    # Chargement
    X, X_const, Y = load_data(x_path, y_path)
    
    # Test 1 : Probit de base
    baseline = test_1_baseline_probit(X_const, Y, report_lines)
    if baseline is None:
        print("\n❌ Arrêt : le modèle de base n'a pas convergé")
        return
    
    # Test 2 : Link Test
    test_2_link_test(X_const, Y, baseline, report_lines)
    
    # Test 3 : Hosmer-Lemeshow
    test_3_hosmer_lemeshow(Y, baseline, report_lines)
    
    # Test 4 : Probit vs Logit
    test_4_probit_vs_logit(X_const, Y, report_lines)
    
    # Test 5 : Robustesse endogénéité
    test_5_endogeneity_robustness(X, Y, baseline, report_lines)
    
    # Test 6 : Sous-échantillons
    test_6_subsamples(X, Y, report_lines)
    
    # Test 7 : Sensibilité
    test_7_sensitivity(X, Y, baseline, report_lines)
    
    # Test 8 : Forme fonctionnelle âge
    test_8_functional_form_age(X, Y, report_lines)
    
    # Générer rapport
    generate_report(report_lines, output_dir)
    
    print("\n" + "=" * 80)
    print("TESTS DE ROBUSTESSE TERMINÉS")
    print("=" * 80)
    
    return baseline


# ============================================================================
# EXÉCUTION
# ============================================================================
if __name__ == "__main__":
    print("\n" + "=" * 80)
    print("TESTS DE ROBUSTESSE - PROBIT TRANSFRONTALIER")
    print("=" * 80)
    
    # Vérifier si les fichiers existent (pour environnement local)
    import os
    if os.path.exists(X_PATH) and os.path.exists(Y_PATH):
        run_all_tests()
    else:
        print(f"\n⚠️ Fichiers non trouvés :")
        print(f"   X : {X_PATH}")
        print(f"   Y : {Y_PATH}")
        print("\n→ Adapter les chemins X_PATH et Y_PATH en haut du script")


TESTS DE ROBUSTESSE - PROBIT TRANSFRONTALIER
CHARGEMENT DES DONNÉES

✓ X chargé : 494,483 obs × 65 variables
✓ Y chargé : 494,483 observations
  → Colonne utilisée : 'TRANSFRONTALIER'

✓ Variable dépendante :
  → Transfrontaliers (Y=1) : 44,264 (8.95%)
  → Non-transfrontaliers   : 450,219 (91.05%)

✓ Constante ajoutée : 66 variables (dont intercept)

TEST 1 : MODÈLE PROBIT DE BASE

✓ Convergence : Oui
✓ Itérations  : 10
✓ Log-L       : -113,380.76
✓ Pseudo R²   : 0.2393
✓ AIC         : 226,893.52
✓ BIC         : 227,626.87
✓ LR test p   : 0.00e+00

--- Coefficients clés ---
  DEPT_57      :   2.4367 (SE=0.0481) ***
  DEPT_68      :   2.0908 (SE=0.0483) ***
  DEPT_54      :   1.9401 (SE=0.0484) ***
  DEPT_67      :   1.4722 (SE=0.0484) ***
  SEXE_2       :  -0.1255 (SE=0.0069) ***
  AGEREV       :   0.0326 (SE=0.0019) ***
  AGEREV_sq    :  -0.0004 (SE=0.0000) ***

TEST 2 : LINK TEST (PREGIBON)

_hat   coef : 1.1247 (p=0.0000)
_hatsq coef : 0.0509 (p=0.0000)

⚠️ H0 rejetée : Possible er

## Interprétation des tests de robustesse 

### Résultats satisfaisants

**Convergence et ajustement global**

Le modèle probit converge rapidement en 10 itérations, avec un pseudo-R² de 0.24, un AIC de 226 894 et un BIC de 227 627. Ces valeurs de pseudo-R² peuvent sembler modestes comparées aux R² de régression linéaire, mais elles sont tout à fait satisfaisantes pour un modèle de choix binaire sur données individuelles. Elles indiquent que les variables retenues captent une part significative de l'hétérogénéité des comportements transfrontaliers, sachant qu'une grande partie des déterminants (préférences, réseaux, compétences linguistiques) n'est pas observable dans les données du recensement.

**Coefficients économiquement cohérents**

Les coefficients départementaux confirment le gradient géographique attendu. La Moselle affiche l'effet le plus fort (+2.44), suivie du Haut-Rhin (+2.09), de la Meurthe-et-Moselle (+1.94) et du Bas-Rhin (+1.47). Cette hiérarchie reflète fidèlement la proximité aux principaux bassins d'emploi frontaliers : Luxembourg pour la Moselle et la Meurthe-et-Moselle, Suisse et Allemagne pour le Haut-Rhin et le Bas-Rhin. L'effet genre est négatif pour les femmes (-0.13), cohérent avec la littérature sur la moindre mobilité professionnelle féminine. L'effet de l'âge suit bien une forme en U inversé, avec un pic d'activité transfrontalière en milieu de carrière.

**Hétérogénéité par sexe**

Les estimations par sous-échantillons révèlent des différences notables entre hommes et femmes. Le taux de transfrontaliers atteint 10.5% chez les hommes contre seulement 7.2% chez les femmes, soit un écart de 3.3 points de pourcentage. Malgré cette différence de prévalence, les pseudo-R² restent comparables : 0.233 pour les hommes et 0.248 pour les femmes. La structure explicative est donc stable entre les deux groupes, même si le niveau diffère. Les estimations par tranche d'âge (moins de 35 ans vs 35 ans et plus) confirment cette stabilité, avec des pseudo-R² de 0.249 et 0.240 respectivement.

**Stabilité des estimations et équivalence Probit/Logit**

La comparaison Probit/Logit montre une corrélation des effets marginaux de 0.995, confirmant l'équivalence pratique des deux spécifications. Les deux modèles produisent des pseudo-R² quasi identiques (0.239 vs 0.240) et des critères d'information très proches. Le choix du probit est donc robuste et n'affecte pas les conclusions substantives.

**Test de robustesse à l'endogénéité**

L'exclusion des 15 variables potentiellement endogènes (nombre de voitures, statut d'occupation du logement, type de logement) permet d'évaluer la sensibilité du modèle à d'éventuels biais de simultanéité. Le modèle réduit, estimé sur 51 variables au lieu de 66, affiche un pseudo-R² de 0.221 contre 0.239 pour le modèle complet. Cette perte de seulement 1.8 point indique que les variables suspectes contribuent peu au pouvoir explicatif global. La log-vraisemblance passe de -113 381 à -116 153, une dégradation modérée. Surtout, les coefficients clés restent remarquablement stables : le coefficient de la Moselle varie de +0.8%, celui du Haut-Rhin de +1.4%, celui du sexe de +4.4% et celui de l'âge de -1.4%. Cette stabilité suggère que les résultats ne sont pas contaminés par un biais d'endogénéité majeur.

**Forme fonctionnelle de l'âge**

Le test du ratio de vraisemblance valide la spécification quadratique de l'âge (χ²=312, p<0.001). L'ajout d'un terme cubique améliore légèrement l'ajustement : le BIC passe de 227 627 à 227 596, soit un gain de 30 points. Ce gain reste marginal et la forme quadratique constitue un bon compromis entre flexibilité et parcimonie. La spécification par splines ne fait pas mieux que la forme cubique, confirmant qu'une paramétrisation polynomiale simple suffit à capturer la relation âge-travail transfrontalier.

---

### Résultats nécessitant approfondissement

**Link Test et Hosmer-Lemeshow rejetés**

Les deux tests classiques de spécification sont rejetés avec des p-values nulles. Le Link Test de Pregibon indique un coefficient de 0.051 pour le terme quadratique (_hatsq), significatif à p<0.001. Le test de Hosmer-Lemeshow produit une statistique de 304.5 à 8 degrés de liberté. Ces résultats pourraient inquiéter, mais ils s'expliquent entièrement par la taille de l'échantillon. À n=494 000, ces tests disposent d'une puissance statistique telle qu'ils détectent la moindre déviation, aussi infime soit-elle, par rapport à l'hypothèse nulle. Un écart de calibration de 0.1 point de pourcentage, sans aucune conséquence pratique, suffit à rejeter H0. Ces tests ne sont donc pas informatifs dans notre contexte et doivent être remplacés par des diagnostics adaptés aux grands échantillons : score de Brier, courbe de calibration par déciles, et AUC avec intervalle de confiance bootstrap.

**Ratio Logit/Probit supérieur à la valeur théorique**

Le ratio moyen des coefficients Logit/Probit s'établit à 2.05, alors que la valeur théorique attendue est d'environ 1.6. Cet écart n'indique pas nécessairement un problème. Il provient vraisemblablement du choix de la Marne comme catégorie de référence. La Marne étant un département non frontalier, les coefficients des départements frontaliers sont mécaniquement très élevés (supérieurs à 2). Or, lorsque les coefficients sont extrêmes, les fonctions de répartition logistique et normale divergent dans les queues de distribution, ce qui amplifie le ratio. Pour vérifier cette hypothèse, il convient de ré-estimer le modèle avec un département frontalier comme référence, par exemple le Bas-Rhin, et d'observer si le ratio se rapproche de 1.6.

**Résidus extrêmes et pseudo-R² sans outliers**

L'analyse identifie 21 432 observations (4.3% de l'échantillon) présentant des résidus standardisés supérieurs à 3 en valeur absolue. Un résultat frappant émerge lorsqu'on exclut ces observations : le pseudo-R² bondit à 0.48, soit un quasi-doublement par rapport au modèle complet. Ce saut spectaculaire indique que ces individus ne sont pas du bruit aléatoire mais une sous-population structurellement différente, dont le comportement échappe aux variables du modèle. 

Il serait tentant de les traiter comme des outliers à exclure pour améliorer l'ajustement, mais cette approche serait méthodologiquement contestable. Ces observations correspondent à des individus réels dont le comportement s'écarte de la prédiction du modèle. Avant toute décision, il est indispensable de caractériser leur profil : sont-ils concentrés géographiquement dans certains départements ? Présentent-ils des caractéristiques sociodémographiques particulières ? S'agit-il principalement de faux négatifs (transfrontaliers non prédits) ou de faux positifs (non-transfrontaliers prédits comme tels) ? Cette analyse permettra de distinguer les éventuelles erreurs de données d'une hétérogénéité comportementale réelle que le modèle ne capture pas.

**Échec des estimations par zone frontalière/intérieure**

L'estimation séparée sur les départements frontaliers et intérieurs a échoué. Ce résultat est normal et attendu : au sein de chaque sous-groupe, les dummies départementales deviennent parfaitement colinéaires (toutes les observations frontalières ont DEPT_57=0 ou 1, etc.), rendant l'estimation impossible. Ce n'est pas un bug mais une conséquence mécanique du découpage.

---

### Tests complémentaires à conduire

Pour répondre aux questions ouvertes, trois analyses complémentaires sont nécessaires. Premièrement, remplacer les tests HL et Link par des diagnostics adaptés aux grands échantillons : score de Brier avec comparaison au modèle nul, courbe de calibration par déciles comparant taux observés et prédits, et AUC-ROC avec intervalle de confiance bootstrap. Deuxièmement, caractériser les résidus extrêmes en analysant leur distribution géographique, leur profil d'âge et de sexe, et la nature des erreurs de prédiction. Troisièmement, tester la sensibilité du ratio Logit/Probit au choix de la catégorie de référence en ré-estimant avec le Bas-Rhin comme référence.

## Tests de robustesse complémentaires

L'estimation initiale du modèle probit a convergé avec un pseudo-R² de 0.24 et des coefficients cohérents économiquement. Cependant, certains résultats appellent une analyse approfondie :

1. **Diagnostics adaptés aux grands échantillons** — Les tests classiques (Hosmer-Lemeshow, Link Test) sont rejetés mécaniquement à n=494k par excès de puissance statistique. On les remplace par : score de Brier, courbe de calibration par déciles, AUC avec intervalle bootstrap.

2. **Caractérisation des résidus extrêmes** — 4.3% d'observations présentent des résidus > 2.5σ. Avant toute décision, on analyse leur distribution géographique et leur profil sociodémographique pour distinguer erreurs de données vs hétérogénéité comportementale réelle.

3. **Sensibilité au choix de la catégorie de référence** — Le ratio Logit/Probit (~2.05) dépasse la valeur théorique (~1.6), possiblement dû aux coefficients extrêmes induits par la Marne (non frontalière) comme référence. On ré-estime avec le Bas-Rhin (frontalier) comme référence pour vérifier.

4. **Comparaison des effets marginaux Probit vs Logit** — Corrélation et écart moyen pour confirmer l'équivalence pratique des deux spécifications.

In [ ]:
"""
================================================================================
TESTS DE ROBUSTESSE  - MODÈLE PROBIT TRANSFRONTALIER
================================================================================
Version adaptée pour grands échantillons (n = 494k)

MODIFICATIONS PAR RAPPORT À V2 :
1. Remplacement HL/Link Test → Brier Score, Calibration, AUC bootstrap
2. Caractérisation des résidus extrêmes (profil, département)
3. Test ratio Logit/Probit avec DEPT_67 comme référence

Catégorie de référence initiale : DEPT_51 (Marne) - non frontalier
Test alternatif : DEPT_67 (Bas-Rhin) comme référence - frontalier
================================================================================
"""

import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
from scipy.stats import chi2
from sklearn.metrics import brier_score_loss, roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.linear_model import LogisticRegression
import warnings
import os
from datetime import datetime

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
X_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/Probit_Ready/BDD_PROBIT.csv"
Y_PATH = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/Probit_Ready/y_final.csv"
OUTPUT_DIR = "/Users/mehdifehri/Desktop/Projet INSEE/BDD/BDD GE/BDD Clean/Probit_Ready/Robustesse_v3"


def load_data(x_path, y_path):
    """Charge les données X et Y"""
    print("=" * 80)
    print("CHARGEMENT DES DONNÉES")
    print("=" * 80)
    
    X = pd.read_csv(x_path)
    print(f"\n✓ X chargé : {X.shape[0]:,} obs × {X.shape[1]} variables")
    
    Y_df = pd.read_csv(y_path)
    y_col = Y_df.columns[0]
    Y = Y_df[y_col].values
    
    n_transfront = Y.sum()
    pct_transfront = 100 * n_transfront / len(Y)
    print(f"✓ Transfrontaliers (Y=1) : {n_transfront:,} ({pct_transfront:.2f}%)")
    
    X_with_const = sm.add_constant(X, has_constant='add')
    
    return X, X_with_const, Y


# ============================================================================
# TEST 1 : MODÈLE DE BASE
# ============================================================================
def test_1_baseline_probit(X, Y, report_lines):
    """Estimation du modèle probit de base"""
    print("\n" + "=" * 80)
    print("TEST 1 : MODÈLE PROBIT DE BASE")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 1 : MODÈLE PROBIT DE BASE")
    report_lines.append("=" * 80)
    
    model = sm.Probit(Y, X)
    result = model.fit(disp=0, maxiter=100)
    
    print(f"\n✓ Convergence : {'Oui' if result.mle_retvals['converged'] else 'Non'}")
    print(f"✓ Log-L       : {result.llf:,.2f}")
    print(f"✓ Pseudo R²   : {result.prsquared:.4f}")
    
    report_lines.append(f"\nConvergence : {'Oui' if result.mle_retvals['converged'] else 'Non'}")
    report_lines.append(f"Log-L       : {result.llf:,.2f}")
    report_lines.append(f"Pseudo R²   : {result.prsquared:.4f}")
    
    # Coefficients clés
    print("\n--- Coefficients clés ---")
    report_lines.append("\n--- Coefficients clés ---")
    
    key_vars = ['DEPT_57', 'DEPT_68', 'DEPT_54', 'DEPT_67', 'SEXE_2', 'AGEREV', 'AGEREV_sq']
    for var in key_vars:
        if var in result.params.index:
            coef = result.params[var]
            se = result.bse[var]
            ci_low = coef - 1.96 * se
            ci_high = coef + 1.96 * se
            print(f"  {var:<12} : {coef:>8.4f} [IC95: {ci_low:.3f} ; {ci_high:.3f}]")
            report_lines.append(f"  {var:<12} : {coef:>8.4f} [IC95: {ci_low:.3f} ; {ci_high:.3f}]")
    
    return result


# ============================================================================
# TEST 2 : DIAGNOSTICS ADAPTÉS AUX GRANDS ÉCHANTILLONS
# ============================================================================
def test_2_diagnostics_grands_echantillons(X, Y, baseline_result, report_lines):
    """
    Diagnostics adaptés aux grands échantillons
    Remplace Hosmer-Lemeshow et Link Test
    """
    print("\n" + "=" * 80)
    print("TEST 2 : DIAGNOSTICS GRANDS ÉCHANTILLONS")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 2 : DIAGNOSTICS GRANDS ÉCHANTILLONS")
    report_lines.append("(Remplace HL et Link Test, inadaptés pour n > 100k)")
    report_lines.append("=" * 80)
    
    y_pred = baseline_result.predict()
    
    # -------------------------------------------------------------------------
    # 2.1 BRIER SCORE
    # -------------------------------------------------------------------------
    print("\n--- 2.1 Brier Score ---")
    report_lines.append("\n--- 2.1 Brier Score ---")
    
    brier = brier_score_loss(Y, y_pred)
    # Brier max pour cette prévalence (benchmark naïf)
    prevalence = Y.mean()
    brier_null = prevalence * (1 - prevalence)  # Prédire toujours la moyenne
    brier_skill = 1 - (brier / brier_null)
    
    print(f"  Brier Score      : {brier:.4f}")
    print(f"  Brier (null)     : {brier_null:.4f}")
    print(f"  Brier Skill Score: {brier_skill:.4f}")
    
    report_lines.append(f"  Brier Score      : {brier:.4f} (plus proche de 0 = meilleur)")
    report_lines.append(f"  Brier null       : {brier_null:.4f} (si on prédit toujours P̄)")
    report_lines.append(f"  Brier Skill      : {brier_skill:.4f} (amélioration vs null, >0 = bon)")
    
    if brier_skill > 0.2:
        print("  ✅ Bon pouvoir prédictif (skill > 0.2)")
        report_lines.append("  ✅ Bon pouvoir prédictif")
    elif brier_skill > 0:
        print("  ✓ Pouvoir prédictif modeste mais positif")
        report_lines.append("  ✓ Pouvoir prédictif modeste")
    else:
        print("  ⚠️ Modèle moins bon que prédiction naïve")
        report_lines.append("  ⚠️ Problème de prédiction")
    
    # -------------------------------------------------------------------------
    # 2.2 COURBE DE CALIBRATION
    # -------------------------------------------------------------------------
    print("\n--- 2.2 Calibration par déciles ---")
    report_lines.append("\n--- 2.2 Calibration par déciles ---")
    
    df_calib = pd.DataFrame({'y': Y, 'pred': y_pred})
    df_calib['decile'] = pd.qcut(df_calib['pred'], 10, labels=False, duplicates='drop')
    
    calib_table = df_calib.groupby('decile').agg({
        'y': ['mean', 'count'],
        'pred': 'mean'
    }).round(4)
    calib_table.columns = ['Obs_rate', 'N', 'Pred_rate']
    calib_table['Écart'] = calib_table['Obs_rate'] - calib_table['Pred_rate']
    calib_table['Écart_pct'] = 100 * calib_table['Écart'] / calib_table['Pred_rate']
    
    print(f"\n  {'Décile':<8} {'N':>8} {'Prédit':>10} {'Observé':>10} {'Écart':>10}")
    print("  " + "-" * 50)
    report_lines.append(f"\n  {'Décile':<8} {'N':>8} {'Prédit':>10} {'Observé':>10} {'Écart':>10}")
    report_lines.append("  " + "-" * 50)
    
    for idx, row in calib_table.iterrows():
        line = f"  {idx:<8} {int(row['N']):>8} {row['Pred_rate']:>10.4f} {row['Obs_rate']:>10.4f} {row['Écart']:>+10.4f}"
        print(line)
        report_lines.append(line)
    
    # Écart moyen absolu de calibration
    mae_calib = calib_table['Écart'].abs().mean()
    print(f"\n  Écart moyen absolu de calibration : {mae_calib:.4f}")
    report_lines.append(f"\n  Écart moyen absolu de calibration : {mae_calib:.4f}")
    
    if mae_calib < 0.02:
        print("  ✅ Excellente calibration")
        report_lines.append("  ✅ Excellente calibration (écart < 2pp)")
    elif mae_calib < 0.05:
        print("  ✓ Calibration acceptable")
        report_lines.append("  ✓ Calibration acceptable (écart < 5pp)")
    else:
        print("  ⚠️ Calibration à améliorer")
        report_lines.append("  ⚠️ Calibration perfectible")
    
    # -------------------------------------------------------------------------
    # 2.3 AUC-ROC AVEC INTERVALLE DE CONFIANCE BOOTSTRAP
    # -------------------------------------------------------------------------
    print("\n--- 2.3 AUC-ROC avec IC Bootstrap ---")
    report_lines.append("\n--- 2.3 AUC-ROC avec IC Bootstrap ---")
    
    auc_point = roc_auc_score(Y, y_pred)
    print(f"  AUC point estimate : {auc_point:.4f}")
    
    # Bootstrap pour IC (100 itérations pour rapidité)
    n_bootstrap = 100
    np.random.seed(42)
    auc_boots = []
    
    for _ in range(n_bootstrap):
        idx = np.random.choice(len(Y), size=len(Y), replace=True)
        y_boot = Y[idx]
        pred_boot = y_pred[idx]
        if len(np.unique(y_boot)) == 2:  # Vérifier qu'on a les deux classes
            auc_boots.append(roc_auc_score(y_boot, pred_boot))
    
    auc_ci_low = np.percentile(auc_boots, 2.5)
    auc_ci_high = np.percentile(auc_boots, 97.5)
    
    print(f"  AUC IC 95%         : [{auc_ci_low:.4f} ; {auc_ci_high:.4f}]")
    report_lines.append(f"  AUC point estimate : {auc_point:.4f}")
    report_lines.append(f"  AUC IC 95%         : [{auc_ci_low:.4f} ; {auc_ci_high:.4f}]")
    
    if auc_point > 0.8:
        print("  ✅ Excellente discrimination (AUC > 0.8)")
        report_lines.append("  ✅ Excellente discrimination")
    elif auc_point > 0.7:
        print("  ✓ Bonne discrimination (AUC > 0.7)")
        report_lines.append("  ✓ Bonne discrimination")
    else:
        print("  ⚠️ Discrimination modeste")
        report_lines.append("  ⚠️ Discrimination modeste")
    
    # -------------------------------------------------------------------------
    # 2.4 VALIDATION CROISÉE 5-FOLD
    # -------------------------------------------------------------------------
    print("\n--- 2.4 Validation croisée 5-fold ---")
    report_lines.append("\n--- 2.4 Validation croisée 5-fold ---")
    
    # Utiliser sklearn LogisticRegression comme proxy (plus rapide)
    # C=1e10 pour pas de régularisation
    X_np = X.values if hasattr(X, 'values') else X
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # AUC par fold
    auc_folds = []
    brier_folds = []
    
    for train_idx, test_idx in cv.split(X_np, Y):
        X_train, X_test = X_np[train_idx], X_np[test_idx]
        y_train, y_test = Y[train_idx], Y[test_idx]
        
        lr = LogisticRegression(C=1e10, max_iter=1000, solver='lbfgs')
        lr.fit(X_train, y_train)
        y_pred_fold = lr.predict_proba(X_test)[:, 1]
        
        auc_folds.append(roc_auc_score(y_test, y_pred_fold))
        brier_folds.append(brier_score_loss(y_test, y_pred_fold))
    
    print(f"  AUC moyen (5-fold)   : {np.mean(auc_folds):.4f} ± {np.std(auc_folds):.4f}")
    print(f"  Brier moyen (5-fold) : {np.mean(brier_folds):.4f} ± {np.std(brier_folds):.4f}")
    
    report_lines.append(f"  AUC moyen (5-fold)   : {np.mean(auc_folds):.4f} ± {np.std(auc_folds):.4f}")
    report_lines.append(f"  Brier moyen (5-fold) : {np.mean(brier_folds):.4f} ± {np.std(brier_folds):.4f}")
    
    # Stabilité
    cv_stability = np.std(auc_folds) / np.mean(auc_folds)
    if cv_stability < 0.02:
        print("  ✅ Très stable entre folds (CV < 2%)")
        report_lines.append("  ✅ Très stable entre folds")
    elif cv_stability < 0.05:
        print("  ✓ Stable entre folds")
        report_lines.append("  ✓ Stable entre folds")
    else:
        print("  ⚠️ Variabilité entre folds")
        report_lines.append("  ⚠️ Variabilité entre folds")
    
    return {
        'brier': brier,
        'brier_skill': brier_skill,
        'mae_calib': mae_calib,
        'auc': auc_point,
        'auc_ci': (auc_ci_low, auc_ci_high),
        'auc_cv': np.mean(auc_folds),
        'calib_table': calib_table
    }


# ============================================================================
# TEST 3 : CARACTÉRISATION DES RÉSIDUS EXTRÊMES
# ============================================================================
def test_3_residus_extremes(X_df, Y, baseline_result, report_lines):
    """
    Analyse et caractérisation des résidus extrêmes
    Objectif : comprendre qui sont les individus mal prédits
    """
    print("\n" + "=" * 80)
    print("TEST 3 : CARACTÉRISATION DES RÉSIDUS EXTRÊMES")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 3 : CARACTÉRISATION DES RÉSIDUS EXTRÊMES")
    report_lines.append("=" * 80)
    
    y_pred = baseline_result.predict()
    
    # Calcul des résidus de Pearson
    residuals_pearson = (Y - y_pred) / np.sqrt(y_pred * (1 - y_pred))
    
    # Résidus standardisés
    residuals_raw = Y - y_pred
    std_res = (residuals_raw - residuals_raw.mean()) / residuals_raw.std()
    
    # -------------------------------------------------------------------------
    # 3.1 DISTRIBUTION DES RÉSIDUS
    # -------------------------------------------------------------------------
    print("\n--- 3.1 Distribution des résidus ---")
    report_lines.append("\n--- 3.1 Distribution des résidus ---")
    
    # Seuils pour définir "extrême"
    seuils = [2, 2.5, 3]
    for seuil in seuils:
        n_extreme = (np.abs(std_res) > seuil).sum()
        pct_extreme = 100 * n_extreme / len(Y)
        print(f"  |résidu| > {seuil}σ : {n_extreme:,} obs ({pct_extreme:.2f}%)")
        report_lines.append(f"  |résidu| > {seuil}σ : {n_extreme:,} obs ({pct_extreme:.2f}%)")
    
    # Utiliser seuil = 2.5 pour l'analyse (compromis)
    seuil_analyse = 2.5
    mask_extreme = np.abs(std_res) > seuil_analyse
    n_extreme = mask_extreme.sum()
    
    print(f"\n  → Analyse détaillée pour |résidu| > {seuil_analyse}σ ({n_extreme:,} obs)")
    report_lines.append(f"\n  → Analyse avec seuil {seuil_analyse}σ : {n_extreme:,} obs")
    
    # -------------------------------------------------------------------------
    # 3.2 TYPES DE RÉSIDUS EXTRÊMES
    # -------------------------------------------------------------------------
    print("\n--- 3.2 Types de résidus extrêmes ---")
    report_lines.append("\n--- 3.2 Types de résidus extrêmes ---")
    
    # Faux négatifs : Y=1 mais P(Y=1) faible
    mask_fn = (Y == 1) & (y_pred < 0.2) & mask_extreme
    n_fn = mask_fn.sum()
    
    # Faux positifs : Y=0 mais P(Y=1) élevée
    mask_fp = (Y == 0) & (y_pred > 0.3) & mask_extreme
    n_fp = mask_fp.sum()
    
    print(f"  Faux négatifs (Y=1, P<0.2) : {n_fn:,}")
    print(f"  Faux positifs (Y=0, P>0.3) : {n_fp:,}")
    
    report_lines.append(f"  Faux négatifs (transfrontaliers non prédits) : {n_fn:,}")
    report_lines.append(f"  Faux positifs (non-transfront. prédits transfr.) : {n_fp:,}")
    
    # -------------------------------------------------------------------------
    # 3.3 RÉPARTITION PAR DÉPARTEMENT
    # -------------------------------------------------------------------------
    print("\n--- 3.3 Répartition géographique ---")
    report_lines.append("\n--- 3.3 Répartition géographique ---")
    
    dept_cols = [c for c in X_df.columns if c.startswith('DEPT_')]
    
    if dept_cols:
        # Reconstruire le département
        dept_df = X_df[dept_cols].copy()
        # Identifier le département de chaque individu
        # Celui qui vaut 1, sinon c'est la référence (51)
        def get_dept(row):
            for col in dept_cols:
                if row[col] == 1:
                    return col.replace('DEPT_', '')
            return '51'  # Référence
        
        dept_series = dept_df.apply(get_dept, axis=1)
        
        # Créer un DataFrame pour l'analyse
        df_analyse = pd.DataFrame({
            'dept': dept_series,
            'Y': Y,
            'pred': y_pred,
            'extreme': mask_extreme
        })
        
        # Statistiques par département
        dept_stats = df_analyse.groupby('dept').agg({
            'Y': ['sum', 'count', 'mean'],
            'extreme': 'sum'
        })
        dept_stats.columns = ['n_transfront', 'n_total', 'taux_transfront', 'n_extreme']
        dept_stats['pct_extreme'] = 100 * dept_stats['n_extreme'] / dept_stats['n_total']
        dept_stats = dept_stats.sort_values('pct_extreme', ascending=False)
        
        print(f"\n  {'Dept':<6} {'N':>8} {'Trans.':>8} {'%Trans':>8} {'Extr.':>8} {'%Extr':>8}")
        print("  " + "-" * 55)
        report_lines.append(f"\n  {'Dept':<6} {'N':>8} {'Trans.':>8} {'%Trans':>8} {'Extr.':>8} {'%Extr':>8}")
        report_lines.append("  " + "-" * 55)
        
        for dept, row in dept_stats.head(10).iterrows():
            line = f"  {dept:<6} {int(row['n_total']):>8} {int(row['n_transfront']):>8} {100*row['taux_transfront']:>7.1f}% {int(row['n_extreme']):>8} {row['pct_extreme']:>7.2f}%"
            print(line)
            report_lines.append(line)
    
    # -------------------------------------------------------------------------
    # 3.4 PROFIL DES RÉSIDUS EXTRÊMES VS POPULATION
    # -------------------------------------------------------------------------
    print("\n--- 3.4 Profil des résidus extrêmes ---")
    report_lines.append("\n--- 3.4 Profil des résidus extrêmes ---")
    
    # Comparer quelques variables clés
    vars_profil = ['AGEREV', 'SEXE_2']
    vars_profil = [v for v in vars_profil if v in X_df.columns]
    
    if vars_profil:
        print(f"\n  {'Variable':<12} {'Pop. totale':>15} {'Extrêmes':>15} {'Diff':>10}")
        print("  " + "-" * 55)
        report_lines.append(f"\n  {'Variable':<12} {'Pop. totale':>15} {'Extrêmes':>15} {'Diff':>10}")
        report_lines.append("  " + "-" * 55)
        
        for var in vars_profil:
            mean_all = X_df[var].mean()
            mean_extreme = X_df.loc[mask_extreme, var].mean()
            diff = mean_extreme - mean_all
            
            line = f"  {var:<12} {mean_all:>15.3f} {mean_extreme:>15.3f} {diff:>+10.3f}"
            print(line)
            report_lines.append(line)
    
    # Taux de transfrontaliers parmi les extrêmes
    taux_transfront_extreme = Y[mask_extreme].mean()
    taux_transfront_total = Y.mean()
    
    print(f"\n  Taux transfrontaliers :")
    print(f"    - Population totale : {100*taux_transfront_total:.2f}%")
    print(f"    - Résidus extrêmes  : {100*taux_transfront_extreme:.2f}%")
    
    report_lines.append(f"\n  Taux transfrontaliers :")
    report_lines.append(f"    - Population totale : {100*taux_transfront_total:.2f}%")
    report_lines.append(f"    - Résidus extrêmes  : {100*taux_transfront_extreme:.2f}%")
    
    # -------------------------------------------------------------------------
    # 3.5 RECOMMANDATION
    # -------------------------------------------------------------------------
    print("\n--- 3.5 Recommandation ---")
    report_lines.append("\n--- 3.5 Recommandation ---")
    
    pct_extreme = 100 * n_extreme / len(Y)
    
    if pct_extreme < 3:
        rec = "✅ Taux d'extrêmes acceptable (<3%). Pas d'action requise."
    elif pct_extreme < 5:
        rec = "✓ Taux d'extrêmes modéré (3-5%). Mentionner dans le rapport, pas de suppression."
    else:
        rec = "⚠️ Taux d'extrêmes élevé (>5%). Investiguer les causes, ne PAS supprimer sans justification."
    
    print(f"  {rec}")
    report_lines.append(f"  {rec}")
    
    # Ne PAS recommander de supprimer sauf erreur de données avérée
    print("\n  IMPORTANT : Les résidus extrêmes ne doivent PAS être supprimés automatiquement.")
    print("  Ce sont de vraies observations qui reflètent l'hétérogénéité de la population.")
    print("  Leur suppression biaiserait les résultats.")
    
    report_lines.append("\n  IMPORTANT : Ne pas supprimer les résidus extrêmes sans justification.")
    report_lines.append("  Ils reflètent l'hétérogénéité naturelle de la population.")
    
    return {
        'n_extreme': n_extreme,
        'pct_extreme': pct_extreme,
        'mask_extreme': mask_extreme
    }


# ============================================================================
# TEST 4 : RATIO LOGIT/PROBIT AVEC CHANGEMENT DE RÉFÉRENCE
# ============================================================================
def test_4_ratio_reference_alternative(X_df, Y, report_lines):
    """
    Test du ratio Logit/Probit en changeant la catégorie de référence
    - Initial : DEPT_51 (Marne) comme référence
    - Alternative : DEPT_67 (Bas-Rhin) comme référence
    
    Hypothèse : le ratio anormal (~2.05 au lieu de ~1.6) vient des coefficients
    extrêmes des départements frontaliers par rapport à la Marne.
    """
    print("\n" + "=" * 80)
    print("TEST 4 : RATIO LOGIT/PROBIT - CHANGEMENT DE RÉFÉRENCE")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 4 : RATIO LOGIT/PROBIT - CHANGEMENT DE RÉFÉRENCE")
    report_lines.append("=" * 80)
    
    # -------------------------------------------------------------------------
    # 4.1 MODÈLE INITIAL (référence = DEPT_51)
    # -------------------------------------------------------------------------
    print("\n--- 4.1 Modèle initial (réf = DEPT_51, Marne) ---")
    report_lines.append("\n--- 4.1 Modèle initial (réf = DEPT_51, Marne) ---")
    
    X_init = sm.add_constant(X_df)
    
    # Probit
    model_probit_init = sm.Probit(Y, X_init)
    result_probit_init = model_probit_init.fit(disp=0)
    
    # Logit
    model_logit_init = sm.Logit(Y, X_init)
    result_logit_init = model_logit_init.fit(disp=0)
    
    # Ratio pour les variables DEPT
    dept_vars = [c for c in X_df.columns if c.startswith('DEPT_')]
    
    print(f"\n  {'Variable':<12} {'Probit':>10} {'Logit':>10} {'Ratio':>10}")
    print("  " + "-" * 45)
    report_lines.append(f"\n  {'Variable':<12} {'Probit':>10} {'Logit':>10} {'Ratio':>10}")
    report_lines.append("  " + "-" * 45)
    
    ratios_init = []
    for var in dept_vars:
        if var in result_probit_init.params.index:
            coef_p = result_probit_init.params[var]
            coef_l = result_logit_init.params[var]
            if abs(coef_p) > 0.01:  # Éviter division par zéro
                ratio = coef_l / coef_p
                ratios_init.append(ratio)
                line = f"  {var:<12} {coef_p:>10.4f} {coef_l:>10.4f} {ratio:>10.3f}"
                print(line)
                report_lines.append(line)
    
    ratio_mean_init = np.mean(ratios_init) if ratios_init else np.nan
    print(f"\n  Ratio moyen (DEPT) : {ratio_mean_init:.3f}")
    report_lines.append(f"\n  Ratio moyen (DEPT) : {ratio_mean_init:.3f}")
    
    # -------------------------------------------------------------------------
    # 4.2 CRÉER NOUVELLE MATRICE AVEC DEPT_67 COMME RÉFÉRENCE
    # -------------------------------------------------------------------------
    print("\n--- 4.2 Reconstruction avec DEPT_67 (Bas-Rhin) comme référence ---")
    report_lines.append("\n--- 4.2 Reconstruction avec DEPT_67 comme référence ---")
    
    # Identifier tous les départements présents
    dept_cols_present = [c for c in X_df.columns if c.startswith('DEPT_')]
    
    if 'DEPT_67' not in dept_cols_present:
        print("  ⚠️ DEPT_67 non trouvé dans les données")
        report_lines.append("  ⚠️ DEPT_67 non trouvé")
        return None
    
    # Créer la nouvelle matrice
    X_new = X_df.copy()
    
    # Récupérer qui est dans le 67
    is_67 = X_new['DEPT_67'] == 1
    
    # Supprimer DEPT_67 (devient la nouvelle référence)
    X_new = X_new.drop(columns=['DEPT_67'])
    
    # Ajouter DEPT_51 (était la référence, maintenant explicite)
    # DEPT_51 = 1 si aucun autre DEPT n'est à 1 (dans la matrice initiale avec 51 comme ref)
    dept_cols_other = [c for c in X_new.columns if c.startswith('DEPT_')]
    is_51 = X_df[dept_cols_present].sum(axis=1) == 0  # Ceux qui n'ont aucun DEPT à 1
    X_new['DEPT_51'] = is_51.astype(int)
    
    # Ajuster pour le 67 : ceux qui étaient 67 ne sont plus dans aucune catégorie
    # (ils deviennent la référence)
    # On doit aussi recoder les autres DEPT pour exclure le 67
    # En fait, on doit reconstruire toute la logique...
    
    # Approche plus simple : reconstruire le département à partir des dummies
    def get_dept_from_dummies(row, dept_cols):
        for col in dept_cols:
            if row[col] == 1:
                return col.replace('DEPT_', '')
        return '51'  # Référence initiale
    
    dept_series = X_df[dept_cols_present].apply(lambda row: get_dept_from_dummies(row, dept_cols_present), axis=1)
    
    # Créer nouvelles dummies avec 67 comme référence
    # Exclure 67, inclure 51
    depts_uniques = dept_series.unique()
    depts_a_encoder = [d for d in depts_uniques if d != '67']
    
    # Supprimer toutes les colonnes DEPT existantes
    X_new = X_df.drop(columns=dept_cols_present)
    
    # Ajouter les nouvelles dummies
    for dept in depts_a_encoder:
        X_new[f'DEPT_{dept}'] = (dept_series == dept).astype(int)
    
    print(f"  Nouvelles dummies DEPT : {[c for c in X_new.columns if c.startswith('DEPT_')][:5]}...")
    report_lines.append(f"  Nouvelles dummies DEPT créées, référence = 67")
    
    # -------------------------------------------------------------------------
    # 4.3 ESTIMATION AVEC NOUVELLE RÉFÉRENCE
    # -------------------------------------------------------------------------
    print("\n--- 4.3 Estimation avec DEPT_67 comme référence ---")
    report_lines.append("\n--- 4.3 Estimation avec DEPT_67 comme référence ---")
    
    X_new_const = sm.add_constant(X_new)
    
    # Probit
    model_probit_new = sm.Probit(Y, X_new_const)
    result_probit_new = model_probit_new.fit(disp=0)
    
    # Logit
    model_logit_new = sm.Logit(Y, X_new_const)
    result_logit_new = model_logit_new.fit(disp=0)
    
    # Ratio pour les variables DEPT
    dept_vars_new = [c for c in X_new.columns if c.startswith('DEPT_')]
    
    print(f"\n  {'Variable':<12} {'Probit':>10} {'Logit':>10} {'Ratio':>10}")
    print("  " + "-" * 45)
    report_lines.append(f"\n  {'Variable':<12} {'Probit':>10} {'Logit':>10} {'Ratio':>10}")
    report_lines.append("  " + "-" * 45)
    
    ratios_new = []
    for var in sorted(dept_vars_new)[:10]:  # Limiter l'affichage
        if var in result_probit_new.params.index:
            coef_p = result_probit_new.params[var]
            coef_l = result_logit_new.params[var]
            if abs(coef_p) > 0.01:
                ratio = coef_l / coef_p
                ratios_new.append(ratio)
                line = f"  {var:<12} {coef_p:>10.4f} {coef_l:>10.4f} {ratio:>10.3f}"
                print(line)
                report_lines.append(line)
    
    ratio_mean_new = np.mean(ratios_new) if ratios_new else np.nan
    
    # -------------------------------------------------------------------------
    # 4.4 COMPARAISON DES RATIOS
    # -------------------------------------------------------------------------
    print("\n--- 4.4 Comparaison ---")
    report_lines.append("\n--- 4.4 Comparaison ---")
    
    print(f"\n  {'Référence':<20} {'Ratio moyen':>15}")
    print("  " + "-" * 40)
    print(f"  {'DEPT_51 (Marne)':<20} {ratio_mean_init:>15.3f}")
    print(f"  {'DEPT_67 (Bas-Rhin)':<20} {ratio_mean_new:>15.3f}")
    print(f"  {'Théorique':<20} {'~1.6':>15}")
    
    report_lines.append(f"\n  {'Référence':<20} {'Ratio moyen':>15}")
    report_lines.append("  " + "-" * 40)
    report_lines.append(f"  {'DEPT_51 (Marne)':<20} {ratio_mean_init:>15.3f}")
    report_lines.append(f"  {'DEPT_67 (Bas-Rhin)':<20} {ratio_mean_new:>15.3f}")
    report_lines.append(f"  {'Théorique':<20} {'~1.6':>15}")
    
    # Interprétation
    print("\n--- 4.5 Interprétation ---")
    report_lines.append("\n--- 4.5 Interprétation ---")
    
    if abs(ratio_mean_new - 1.6) < abs(ratio_mean_init - 1.6):
        conclusion = """
  ✅ HYPOTHÈSE CONFIRMÉE : Le ratio se rapproche de 1.6 avec la nouvelle référence.
  
  Explication : Avec DEPT_51 (non frontalier) comme référence, les coefficients
  des départements frontaliers (57, 68) étaient très élevés (~2.5), car ils 
  captent TOUT l'écart avec un département sans frontière.
  
  Ces coefficients extrêmes amplifient la différence entre les fonctions 
  logistique et normale dans les queues de distribution, d'où un ratio > 1.6.
  
  Avec DEPT_67 comme référence (frontalier), les coefficients sont plus modérés
  et le ratio revient vers sa valeur théorique.
  
  → Ce n'est PAS un problème du modèle, c'est une conséquence du design 
    (catégorie de référence non frontalière).
  → Le modèle avec DEPT_51 comme référence reste valide et préférable pour
    l'interprétation économique (effet frontière direct)."""
    else:
        conclusion = """
  ⚠️ HYPOTHÈSE NON CONFIRMÉE : Le ratio reste élevé même avec la nouvelle référence.
  
  Causes possibles :
  - Séparation quasi-parfaite sur certaines modalités
  - Non-linéarités non captées
  - À investiguer plus avant"""
    
    print(conclusion)
    report_lines.append(conclusion)
    
    return {
        'ratio_init': ratio_mean_init,
        'ratio_new': ratio_mean_new,
        'result_probit_new': result_probit_new,
        'result_logit_new': result_logit_new
    }


# ============================================================================
# TEST 5 : PROBIT VS LOGIT (effets marginaux)
# ============================================================================
def test_5_probit_vs_logit_mfx(X, Y, report_lines):
    """Comparaison Probit vs Logit via effets marginaux"""
    print("\n" + "=" * 80)
    print("TEST 5 : PROBIT VS LOGIT (EFFETS MARGINAUX)")
    print("=" * 80)
    
    report_lines.append("\n" + "=" * 80)
    report_lines.append("TEST 5 : PROBIT VS LOGIT (EFFETS MARGINAUX)")
    report_lines.append("=" * 80)
    
    X_const = sm.add_constant(X)
    
    # Probit
    result_probit = sm.Probit(Y, X_const).fit(disp=0)
    mfx_probit = result_probit.get_margeff(at='mean').margeff
    
    # Logit
    result_logit = sm.Logit(Y, X_const).fit(disp=0)
    mfx_logit = result_logit.get_margeff(at='mean').margeff
    
    # Corrélation
    corr_mfx = np.corrcoef(mfx_probit, mfx_logit)[0, 1]
    
    # Différence moyenne absolue
    mae_mfx = np.mean(np.abs(mfx_probit - mfx_logit))
    
    print(f"\n  Corrélation effets marginaux : {corr_mfx:.6f}")
    print(f"  Différence moyenne absolue   : {mae_mfx:.6f}")
    
    report_lines.append(f"\n  Corrélation effets marginaux : {corr_mfx:.6f}")
    report_lines.append(f"  Différence moyenne absolue   : {mae_mfx:.6f}")
    
    if corr_mfx > 0.999:
        print("\n  ✅ Probit et Logit donnent des résultats quasi-identiques")
        print("     → Le choix du lien n'affecte pas les conclusions")
        report_lines.append("\n  ✅ Choix du lien sans impact sur les conclusions")
    elif corr_mfx > 0.99:
        print("\n  ✓ Probit et Logit très similaires")
        report_lines.append("\n  ✓ Probit et Logit très similaires")
    else:
        print("\n  ⚠️ Différences notables entre Probit et Logit")
        report_lines.append("\n  ⚠️ Différences notables")
    
    return {
        'corr_mfx': corr_mfx,
        'mae_mfx': mae_mfx
    }


# ============================================================================
# GÉNÉRATION DU RAPPORT
# ============================================================================
def generate_report(report_lines, output_dir):
    """Génère le rapport final"""
    os.makedirs(output_dir, exist_ok=True)
    
    header = [
        "=" * 80,
        "RAPPORT DE ROBUSTESSE V3 - MODÈLE PROBIT TRANSFRONTALIER",
        f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        "=" * 80,
        "",
        "AMÉLIORATIONS V3 :",
        "- Remplacement HL/Link Test → Brier, Calibration, AUC bootstrap",
        "- Caractérisation des résidus extrêmes",
        "- Test ratio avec changement de catégorie de référence",
        ""
    ]
    
    full_report = header + report_lines
    
    report_path = os.path.join(output_dir, "rapport_robustesse_v3.txt")
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(full_report))
    
    print(f"\n✓ Rapport sauvegardé : {report_path}")
    return report_path


# ============================================================================
# EXÉCUTION PRINCIPALE
# ============================================================================
def run_all_tests(x_path=X_PATH, y_path=Y_PATH, output_dir=OUTPUT_DIR):
    """Exécute tous les tests"""
    report_lines = []
    
    # Chargement
    X_df, X_const, Y = load_data(x_path, y_path)
    
    # Test 1 : Probit de base
    baseline = test_1_baseline_probit(X_const, Y, report_lines)
    
    # Test 2 : Diagnostics grands échantillons
    test_2_diagnostics_grands_echantillons(X_const, Y, baseline, report_lines)
    
    # Test 3 : Caractérisation résidus extrêmes
    test_3_residus_extremes(X_df, Y, baseline, report_lines)
    
    # Test 4 : Ratio avec changement de référence
    test_4_ratio_reference_alternative(X_df, Y, report_lines)
    
    # Test 5 : Probit vs Logit (effets marginaux)
    test_5_probit_vs_logit_mfx(X_df, Y, report_lines)
    
    # Générer rapport
    generate_report(report_lines, output_dir)
    
    print("\n" + "=" * 80)
    print("TESTS DE ROBUSTESSE V3 TERMINÉS")
    print("=" * 80)
    
    return baseline


if __name__ == "__main__":
    import os
    if os.path.exists(X_PATH) and os.path.exists(Y_PATH):
        run_all_tests()
    else:
        print(f"\n⚠️ Fichiers non trouvés. Adapter X_PATH et Y_PATH.")

CHARGEMENT DES DONNÉES

✓ X chargé : 494,483 obs × 65 variables
✓ Transfrontaliers (Y=1) : 44,264 (8.95%)

TEST 1 : MODÈLE PROBIT DE BASE

✓ Convergence : Oui
✓ Log-L       : -113,380.76
✓ Pseudo R²   : 0.2393

--- Coefficients clés ---
  DEPT_57      :   2.4367 [IC95: 2.342 ; 2.531]
  DEPT_68      :   2.0908 [IC95: 1.996 ; 2.185]
  DEPT_54      :   1.9401 [IC95: 1.845 ; 2.035]
  DEPT_67      :   1.4722 [IC95: 1.377 ; 1.567]
  SEXE_2       :  -0.1255 [IC95: -0.139 ; -0.112]
  AGEREV       :   0.0326 [IC95: 0.029 ; 0.036]
  AGEREV_sq    :  -0.0004 [IC95: -0.000 ; -0.000]

TEST 2 : DIAGNOSTICS GRANDS ÉCHANTILLONS

--- 2.1 Brier Score ---
  Brier Score      : 0.0681
  Brier (null)     : 0.0815
  Brier Skill Score: 0.1643
  ✓ Pouvoir prédictif modeste mais positif

--- 2.2 Calibration par déciles ---

  Décile          N     Prédit    Observé      Écart
  --------------------------------------------------
  0           49449     0.0001     0.0004    +0.0003
  1           49448     0.0008  

## Interprétation des tests de robustesse complémentaires

### Diagnostics grands échantillons : modèle bien calibré et discriminant

**Score de Brier et pouvoir prédictif**

Le score de Brier s'établit à 0.068, contre 0.082 pour un modèle naïf prédisant systématiquement la moyenne. Le Brier Skill Score de 0.16 indique une amélioration de 16% par rapport au modèle nul. Ce gain peut sembler modeste, mais il est cohérent avec la nature du problème : prédire un comportement individuel rare (9% de transfrontaliers) à partir de caractéristiques sociodémographiques est intrinsèquement difficile. Le modèle apporte une information réelle, même si une part importante de l'hétérogénéité échappe aux variables disponibles.

**Calibration par déciles**

La calibration est excellente. L'écart moyen absolu entre taux prédits et taux observés n'est que de 0.33 point de pourcentage. Pour chaque décile de probabilité prédite, le taux observé de transfrontaliers correspond quasi parfaitement à la prédiction. Par exemple, dans le dernier décile (probabilités prédites autour de 36%), on observe effectivement 35.6% de transfrontaliers. Cette précision confirme que le modèle probit ne souffre d'aucun biais systématique de sur- ou sous-estimation, contrairement à ce que suggéraient les tests HL et Link rejetés dans la version précédente.

**Discrimination (AUC-ROC)**

L'AUC atteint 0.849 avec un intervalle de confiance bootstrap très resserré [0.848 ; 0.850]. Une AUC supérieure à 0.8 est généralement considérée comme excellente. Le modèle distingue efficacement les transfrontaliers des non-transfrontaliers : un individu tiré aléatoirement parmi les transfrontaliers aura, dans 85% des cas, une probabilité prédite supérieure à celle d'un individu tiré parmi les non-transfrontaliers.

**Stabilité en validation croisée**

La validation croisée 5-fold confirme la robustesse des résultats. L'AUC moyen (0.841) et le Brier moyen (0.069) sont très proches des valeurs sur l'échantillon complet, avec des écarts-types inférieurs à 1%. Le modèle ne souffre pas de sur-ajustement et ses performances se généralisent bien à des données non vues.

---

### Résidus extrêmes : des transfrontaliers atypiques, pas des erreurs

**Nature des résidus extrêmes**

L'analyse révèle que 36 005 observations (7.3%) présentent des résidus supérieurs à 2.5 écarts-types. Le résultat le plus frappant est leur composition : 99.3% sont des transfrontaliers. Plus précisément, on compte 19 332 faux négatifs (transfrontaliers avec probabilité prédite inférieure à 20%) contre seulement 244 faux positifs. Le modèle ne "sur-prédit" pratiquement jamais le statut transfrontalier. En revanche, il échoue à identifier une partie significative des transfrontaliers effectifs.

**Concentration géographique**

Ces résidus extrêmes ne sont pas distribués aléatoirement. Ils se concentrent massivement dans les départements à fort taux de transfrontaliers : Moselle (15.5% d'extrêmes), Haut-Rhin (12.2%), Meurthe-et-Moselle (8.5%). Dans les départements non frontaliers, le taux d'extrêmes est négligeable (moins de 0.2%). Cette concentration suggère que le modèle capture bien la structure générale du phénomène mais peine à expliquer pourquoi, parmi les résidents des zones frontalières, certains deviennent transfrontaliers et d'autres non.

**Profil des individus mal prédits**

Les transfrontaliers mal prédits sont légèrement plus âgés que la moyenne (+0.5 an) et comptent proportionnellement moins de femmes (41% contre 48% dans la population totale). Ces écarts sont faibles, ce qui suggère que les caractéristiques discriminantes échappent aux variables du modèle. Il pourrait s'agir de facteurs non observés : compétences linguistiques, réseaux professionnels transfrontaliers, historique familial de mobilité, ou simplement préférences individuelles.

**Recommandation**

Ces observations ne doivent pas être supprimées. Elles représentent des individus réels dont le comportement s'écarte de la prédiction moyenne. Leur exclusion biaiserait artificiellement les résultats à la hausse et réduirait la validité externe du modèle. L'existence de ces résidus reflète une limite structurelle : les variables disponibles dans le recensement ne suffisent pas à expliquer toute l'hétérogénéité des choix de mobilité transfrontalière.

---

### Ratio Logit/Probit : hypothèse confirmée

**Résultat du test**

Avec la Marne (non frontalière) comme référence, le ratio moyen des coefficients Logit/Probit s'établissait à 2.91, bien au-delà de la valeur théorique de 1.6. Après ré-estimation avec le Bas-Rhin (frontalier) comme référence, ce ratio descend à 2.37. La diminution est substantielle, même si le ratio reste supérieur à 1.6.

**Interprétation**

L'hypothèse est confirmée : l'écart au ratio théorique provient bien des coefficients extrêmes induits par le choix d'une référence non frontalière. Avec la Marne comme référence, les coefficients de la Moselle et du Haut-Rhin dépassent 2.4, captant l'intégralité de l'écart entre un département sans frontière et les zones les plus transfrontalières. À ces valeurs élevées, les fonctions de répartition logistique et normale divergent significativement dans les queues de distribution, amplifiant mécaniquement le ratio.

Avec le Bas-Rhin comme référence, les coefficients sont plus modérés. La Moselle affiche +0.96 (écart relatif entre deux départements frontaliers) au lieu de +2.44 (écart absolu avec un département intérieur). Le ratio se rapproche alors de sa valeur théorique, sans l'atteindre complètement en raison de la persistance de quelques coefficients élevés pour les départements non frontaliers.

**Conclusion méthodologique**

Ce résultat ne remet pas en cause la validité du modèle avec la Marne comme référence. Au contraire, cette spécification reste préférable pour l'interprétation économique car elle mesure directement l'effet d'être frontalier par rapport à ne pas l'être. L'écart au ratio théorique est une conséquence attendue du design, non un signe de mauvaise spécification.

---

### Équivalence Probit/Logit confirmée

La corrélation des effets marginaux entre les deux spécifications atteint 0.995, avec une différence moyenne absolue de 0.5 point de pourcentage. Les conclusions substantives seraient rigoureusement identiques quel que soit le modèle retenu. Le choix du probit, justifié par sa cohérence avec le modèle bivarié prévu pour l'analyse des couples, n'introduit aucun biais par rapport à l'alternative logit.

# Lecture statistique du Modèle Probit : Déterminants du travail transfrontalier dans le Grand Est


Ce rapport présente l'analyse économétrique des déterminants individuels de la probabilité d'être travailleur transfrontalier dans la région Grand Est. L'estimation repose sur un modèle **Probit binaire** appliqué à **494 483 observations** issues du Recensement de la Population 2022 (INSEE) et de la base de donnée MOBPRO des mobilités professionnelles.

**Résultat principal** : La probabilité moyenne d'être transfrontalier est de **8,95%** dans la population étudiée. La localisation géographique (proximité frontalière) constitue le déterminant le plus puissant, suivie par la nationalité, le secteur d'activité et les caractéristiques socioprofessionnelles.

### Rappel : Convention de lecture (à appliquer dans toutes les sections)

Pour chaque variable significative, les résultats sont présentés systématiquement sous deux formes :

- **En points de pourcentage** : "Toutes choses égales par ailleurs, [variable] augmente/diminue la probabilité d'être transfrontalier de X points de pourcentage par rapport à [référence]."

- **En odds ratio** : "Les chances d'être transfrontalier sont multipliées par Y / réduites de Z % pour [catégorie] par rapport à [référence]."

Cette double lecture permet de satisfaire les attentes des économistes (effets marginaux interprétables) comme des praticiens (odds ratios familiers en épidémiologie et sciences sociales).

### Classification des variables selon le risque d'endogénéité

Les variables explicatives sont classées selon leur statut causal :

| Statut | Interprétation | Variables concernées |
|--------|----------------|----------------------|
| **Exogène** | Relation causale plausible | Département, âge, sexe, nationalité, lieu de naissance, diplôme, enfants, secteur, CSP |
| **Ambigu** | Causalité incertaine | Type de contrat, temps partiel |
| **Endogène** | Association statistique (causalité inverse possible) | Propriétaire, nombre de voitures, type de logement, superficie |

---

## 1. Effets géographiques : la proximité frontalière comme déterminant majeur

> **Variables exogènes** : Le département de résidence est généralement choisi avant l'emploi ou déterminé par des facteurs familiaux. L'interprétation causale est justifiée.

**Catégorie de référence** : Marne (51) — département intérieur sans frontière directe

| Département             | Effet marginal | Odds Ratio | Frontière principale |
|-------------            |:--------------:|:----------:|----------------------|
| Moselle (57)            | +15,35 pp***   | 4,71       | Luxembourg |
| Haut-Rhin (68)          | +13,17 pp***   | 3,87       | Suisse / Allemagne |
| Meurthe-et-Moselle (54) | +12,22 pp***   | 3,56       | Luxembourg |
| Meuse (55)              | +9,59 pp***    | 2,82       | Belgique / Luxembourg |
| Bas-Rhin (67)           | +9,27 pp***    | 2,73       | Allemagne |
| Vosges (88)             | +1,85 pp***    | 1,30       | (éloigné des frontières) |
| Aube (10)               | +0,66 pp       | 1,11       | Intérieur (NS) |
| Haute-Marne (52)        | +0,75 pp       | 1,12       | Intérieur (NS) |

*Note : pp = points de pourcentage ; OR = Odds Ratio ; NS = non significatif ; \*\*\* p < 0.001*

### Lecture des résultats

**En points de pourcentage :**
- Toutes choses égales par ailleurs, **résider en Moselle plutôt que dans la Marne augmente la probabilité d'être travailleur transfrontalier de 15,35 points de pourcentage**.
- **Résider dans le Haut-Rhin augmente la probabilité d'être transfrontalier de 13,17 points de pourcentage** par rapport à la Marne.
- **Résider en Meurthe-et-Moselle augmente la probabilité d'être transfrontalier de 12,22 points de pourcentage** par rapport à la Marne.

**En odds ratio :**
- **Les chances d'être transfrontalier sont multipliées par 4,7 pour un résident de Moselle** par rapport à un résident de la Marne.
- **Les chances sont multipliées par 3,9 pour un résident du Haut-Rhin** par rapport à la Marne.
- **Les chances sont multipliées par 3,6 pour un résident de Meurthe-et-Moselle** par rapport à la Marne.

### Interprétation économique

La hiérarchie des effets géographiques reflète fidèlement l'attractivité différenciée des marchés du travail frontaliers :

1. **Luxembourg** (Moselle, Meurthe-et-Moselle, Meuse) : salaires élevés, fiscalité avantageuse, forte demande de main-d'œuvre
2. **Suisse** (Haut-Rhin) : rémunérations très attractives dans l'industrie et les services
3. **Allemagne** (Bas-Rhin, Haut-Rhin) : proximité industrielle et complémentarité des marchés du travail

Les départements intérieurs (Aube, Haute-Marne) ne présentent pas de différence significative avec la Marne, confirmant que l'effet frontalier est le moteur principal.

---

## 2. Nationalité et origine géographique : l'effet des réseaux transnationaux

> **Variables exogènes** : La nationalité et le lieu de naissance sont déterminés avant l'entrée sur le marché du travail. L'interprétation causale est justifiée.

**Catégorie de référence** : Français né dans le Grand Est

| Variable                                | Effet marginal | Odds Ratio |
|----------                               |:--------------:|:----------:|
| Étranger (vs Français)                  | +3,12 pp***    | 1,56 |
| Né à l'étranger (vs né Grand Est)       | +2,20 pp***    | 1,40 |
| Né ailleurs en France (vs né Grand Est) | -0,19 pp**     | 0,97 |

### Lecture des résultats

**En points de pourcentage :**
- Toutes choses égales par ailleurs, **être de nationalité étrangère augmente la probabilité d'être transfrontalier de 3,12 points de pourcentage** par rapport aux Français.
- **Être né à l'étranger augmente la probabilité d'être transfrontalier de 2,20 points de pourcentage** par rapport aux personnes nées dans le Grand Est.
- **Être né dans une autre région française diminue la probabilité de 0,19 point de pourcentage**, suggérant un moindre ancrage dans les dynamiques frontalières locales.

**En odds ratio :**
- **Les chances d'être transfrontalier sont multipliées par 1,56 pour les étrangers** par rapport aux Français.
- **Les chances sont multipliées par 1,40 pour les personnes nées à l'étranger** par rapport à celles nées dans le Grand Est.

### Interprétation économique

Ces résultats mettent en évidence le rôle des **réseaux transnationaux** dans l'accès au travail frontalier :

- Les étrangers disposent souvent de **connexions familiales ou communautaires** dans les pays voisins facilitant l'information et l'embauche
- Les personnes nées à l'étranger possèdent fréquemment des **compétences linguistiques** (allemand, luxembourgeois, anglais) valorisées sur les marchés frontaliers
- Une **moindre aversion à la mobilité internationale** peut également jouer

---

## 3. Profil démographique

### 3.1 Effet de l'âge : une relation en U inversé

> **Variable exogène** : L'âge est une caractéristique individuelle fixe. L'interprétation causale est justifiée.

| Variable           | Effet marginal |
|----------          |:--------------:|
| Âge (linéaire)     | +0,21 pp par année*** |
| Âge² (quadratique) | -0,0025 pp par année²*** |

**Âge optimal calculé** : ≈ **41 ans**

### Lecture des résultats

- Toutes choses égales par ailleurs, **chaque année d'âge supplémentaire augmente la probabilité d'être transfrontalier de 0,21 point de pourcentage** jusqu'à environ 41 ans.
- **Au-delà de 41 ans, l'effet s'inverse** et la probabilité décroît progressivement.

### Interprétation économique

La relation en U inversé reflète deux dynamiques opposées :

- **Phase ascendante (< 41 ans)** : accumulation d'expérience professionnelle, stabilisation familiale permettant d'assumer les contraintes de mobilité, progression salariale rendant les trajets rentables
- **Phase descendante (> 41 ans)** : pénibilité croissante des trajets quotidiens, ancrage territorial renforcé, arbitrage en faveur de la qualité de vie

---

### 3.2 Effet du genre

> **Variable exogène** : Le sexe est une caractéristique individuelle fixe. L'interprétation causale est justifiée.

**Catégorie de référence** : Homme

| Variable | Effet marginal | Odds Ratio |
|----------|:--------------:|:----------:|
| Femme    | -0,79 pp***    | 0,88       |

### Lecture des résultats

- Toutes choses égales par ailleurs, **être une femme diminue la probabilité d'être transfrontalier de 0,79 point de pourcentage** par rapport aux hommes.
- **Les chances d'être transfrontalier sont réduites de 12% pour les femmes** (OR = 0,88) par rapport aux hommes.

### Interprétation économique

Ce différentiel de genre, cohérent avec la littérature sur les mobilités professionnelles, peut s'expliquer par :

1. **La charge familiale** : les femmes assurent encore majoritairement les responsabilités domestiques et parentales, peu compatibles avec les longs trajets
2. **La segmentation sectorielle** : surreprésentation féminine dans le secteur public et la santé, structurellement non-transfrontaliers

---

## 4. Situation familiale : la contrainte parentale

> **Variables exogènes** : Le nombre d'enfants et la structure familiale ne sont pas directement causés par le statut de travailleur transfrontalier. L'interprétation causale est justifiée.

**Catégorie de référence** : Sans enfant (NENFR = 0)

| Nombre d'enfants     | Effet marginal | Odds Ratio |
|------------------    |:--------------:|:----------:|
| 1 enfant             | -0,45 pp***     | 0,94 |
| 2 enfants            | -0,59 pp***     | 0,92 |
| 3 enfants            | -1,18 pp***     | 0,85 |
| 4 enfants ou plus    | -1,34 pp***     | 0,83 |
| Hors famille (isolé) | +0,52 pp***     | 1,07 |

| Statut conjugal      | Effet marginal | Odds Ratio |
|-----------------     |:--------------:|:----------:|
| Ne vit pas en couple | +0,14 pp*      | 1,02       |

*Note : \* p < 0.05*

### Lecture des résultats

- Toutes choses égales par ailleurs, **avoir un enfant diminue la probabilité d'être transfrontalier de 0,45 point de pourcentage** par rapport aux personnes sans enfant.
- **Avoir trois enfants diminue la probabilité d'être transfrontalier de 1,18 point de pourcentage** par rapport aux personnes sans enfant.
- **Avoir quatre enfants ou plus diminue la probabilité d'être transfrontalier de 1,34 point de pourcentage** par rapport aux personnes sans enfant.
- **Les personnes hors structure familiale voient leur probabilité augmenter de 0,52 point de pourcentage**.

### Interprétation économique

L'effet est **monotone décroissant** : chaque enfant supplémentaire réduit la probabilité de travail transfrontalier. En moyenne, l'effet marginal de chaque enfant additionnel est d'environ **0,3 à 0,4 point de pourcentage**, avec une accélération au-delà de deux enfants. Cette relation s'explique par :

- **L'incompatibilité temporelle** : trajets longs (souvent 1h-1h30 par trajet) difficilement conciliables avec les horaires scolaires et la garde d'enfants
- **Les coûts indirects** : garde d'enfants, activités extrascolaires nécessitant une présence parentale
- **L'arbitrage familial** : dans les couples, un des conjoints peut renoncer au travail frontalier pour assurer la logistique familiale

Le statut conjugal en lui-même (vivre en couple ou non) n'a qu'un effet marginal et faiblement significatif (p = 0.022), confirmant que c'est bien la **présence d'enfants**, et non le couple en soi, qui constitue la contrainte déterminante.

---

## 5. Caractéristiques professionnelles

### 5.1 Catégorie socioprofessionnelle

> **Variable exogène** : La CSP reflète une qualification et une trajectoire professionnelle préexistantes. L'interprétation causale est justifiée.

**Catégorie de référence** : Employés (GS_5)

| CSP                                       | Effet marginal | Odds Ratio |
|-----                                      |:--------------:|:----------:|
| Ouvriers                                  | +0,59 pp***    | 1,08 |
| Professions intermédiaires                | -0,25 pp***    | 0,97 |
| Cadres                                    | -0,53 pp***    | 0,93 |
| Artisans, commerçants, chefs d'entreprise | -1,93 pp***    | 0,77 |
| Agriculteurs                              | -0,68 pp       | 0,91 (NS) |

### Lecture des résultats

- Toutes choses égales par ailleurs, **être ouvrier augmente la probabilité d'être transfrontalier de 0,59 point de pourcentage** par rapport aux employés.
- **Être artisan, commerçant ou chef d'entreprise diminue la probabilité d'être transfrontalier de 1,93 point de pourcentage** par rapport aux employés.
- **Être cadre diminue la probabilité d'être transfrontalier de 0,53 point de pourcentage** par rapport aux employés.

### Interprétation économique

Les **ouvriers** constituent la catégorie la plus encline au travail transfrontalier, ce qui reflète :

- La **forte demande industrielle** au Luxembourg et en Allemagne (automobile, métallurgie, construction)
- Des **différentiels salariaux particulièrement attractifs** pour les qualifications ouvrières
- La **tradition industrielle transfrontalière** du bassin lorrain et alsacien

Les indépendants (artisans, commerçants) présentent logiquement la probabilité la plus faible, leur activité étant par nature **ancrée territorialement** (clientèle locale, établissement commercial).

---

### 5.2 Type de contrat et statut d'emploi

> **Variables ambiguës** : Le type de contrat peut être une condition d'accès au travail frontalier (exogène) ou une conséquence de l'emploi frontalier (endogène). L'interprétation reste prudente.

**Catégorie de référence** : CDI / Titulaire de la fonction publique (EMPL_16)

| Type de contrat       | Effet marginal | Odds Ratio |
|-----------------      |:--------------:|:----------:|
| Stagiaires rémunérés  | +2,61 pp***    | 1,49 |
| Intérim               | -0,49 pp***    | 0,93 |
| CDD, contrats courts  | -0,63 pp***    | 0,91 |
| Employeurs            | -2,98 pp***    | 0,69 |
| Apprentissage         | -3,33 pp***    | 0,64 |
| Indépendants          | -3,58 pp***    | 0,62 |
| Emplois aidés         | -4,70 pp***    | 0,52 |
| Aides familiaux       | -1,40 pp       | 0,82 (NS, p=0.069) |

### Lecture des résultats

- Les personnes en CDI ou titulaires de la fonction publique constituent la **norme du travail transfrontalier**.
- **Être stagiaire rémunéré est associé à une probabilité supérieure de 2,61 points de pourcentage**. Cet effet, a priori surprenant, s'explique par plusieurs mécanismes :
  - Les **formations transfrontalières** (cursus franco-allemands, masters européens) imposent souvent des stages dans le pays partenaire
  - Le Luxembourg mène une **politique active d'accueil de stagiaires** français, notamment dans le secteur financier et les institutions européennes
  - Les grandes entreprises frontalières (ArcelorMittal, banques luxembourgeoises) disposent de programmes de stages structurés ciblant les étudiants du Grand Est
- **Être en emploi aidé est associé à une probabilité inférieure de 4,70 points de pourcentage** par rapport aux CDI.
- **Être indépendant est associé à une probabilité inférieure de 3,58 points de pourcentage** par rapport aux CDI.
- **Être en apprentissage est associé à une probabilité inférieure de 3,33 points de pourcentage** par rapport aux CDI. Cette différence avec les stagiaires rémunérés s'explique par l'ancrage territorial des CFA et l'obligation de présence en entreprise locale.

### Interprétation économique

Le travail transfrontalier est fortement associé à la **stabilité contractuelle** (CDI) pour plusieurs raisons :

- **Amortissement des coûts fixes** : l'investissement initial (trajet, éventuellement déménagement, adaptation) n'est rentable que sur le long terme
- **Accès au crédit** : les revenus frontaliers (souvent élevés) permettent l'accession à la propriété, conditionnée à un emploi stable
- **Sélection par les employeurs** : les entreprises luxembourgeoises et allemandes privilégient les CDI

Les formes précaires (emplois aidés, apprentissage, intérim) sont sous-représentées car elles concernent des populations éloignées des prérequis du travail frontalier (mobilité, qualification, stabilité).

---

### 5.3 Secteur d'activité

> **Variable exogène** : Le secteur d'activité détermine structurellement la possibilité d'être transfrontalier. L'interprétation causale est justifiée.

**Catégorie de référence** : Commerce, transports et services divers (NA5_GU)

| Secteur                             | Effet marginal | Odds Ratio |
|---------                            |:--------------:|:----------:|
| Industrie                           | +0,08 pp       | 1,01 (NS) |
| Construction                        | -0,13 pp       | 0,98 (NS, p=0.08) |
| Administration, enseignement, santé | -4,16 pp***    | 0,55 |
| Agriculture                         | -6,01 pp***    | 0,44 |

### Lecture des résultats

- Toutes choses égales par ailleurs, **travailler dans l'administration publique, l'enseignement ou la santé diminue la probabilité d'être transfrontalier de 4,16 points de pourcentage** par rapport au secteur tertiaire marchand.
- **Travailler dans l'agriculture diminue la probabilité d'être transfrontalier de 6,01 points de pourcentage** par rapport au secteur tertiaire marchand.
- **Les chances d'être transfrontalier sont réduites de 45% dans le secteur public** (OR = 0,55) et de **56% dans l'agriculture** (OR = 0,44).
- **L'industrie et la construction ne diffèrent pas significativement** du secteur tertiaire marchand.

### Interprétation économique

Le secteur public (administration, enseignement, santé) est **structurellement non-transfrontalier** :

- Ces emplois sont liés au **territoire national** (fonctionnaires, hospitaliers, enseignants)
- Les systèmes de santé et d'éducation ne sont pas intégrés entre pays
- Les concours et recrutements sont nationaux

L'agriculture, par nature **ancrée géographiquement** (exploitation, terres), exclut de facto la mobilité transfrontalière.

---

## 6. Capital humain : une relation non linéaire avec le diplôme

> **Variable exogène** : Le diplôme est obtenu avant l'entrée sur le marché du travail. L'interprétation causale est justifiée.

**Catégorie de référence** : CAP, BEP ou diplôme équivalent (DIPL_13)

| Niveau de diplôme                  | Effet marginal | Odds Ratio |
|-------------------                 |:--------------:|:----------:|
| Doctorat de recherche              | +1,54 pp***    | 1,21 |
| Master, Bac+5, Grande école        | +1,40 pp***    | 1,19 |
| CEP (certificat d'études primaires)| +0,77 pp***    | 1,10 |
| Licence, Bac+3/4                   | +0,50 pp***    | 1,07 |
| BEPC, Brevet                       | -0,14 pp       | 0,98 (NS) 
| Bac général/technologique          | -0,31 pp***    | 0,96 |
| Bac professionnel                  | -0,22 pp**     | 0,97 |
| BTS, DUT, Bac+2                    | -0,31 pp***    | 0,96 |
| Aucun diplôme (scolarité collège)  | -0,61 pp***    | 0,92 |
| Aucun diplôme (scolarité primaire) | -1,60 pp***    | 0,80 |
| Pas de scolarité                   | -3,28 pp***    | 0,62 |

### Lecture des résultats

- Toutes choses égales par ailleurs, **être titulaire d'un doctorat augmente la probabilité d'être transfrontalier de 1,54 point de pourcentage** par rapport aux titulaires d'un CAP/BEP.
- **Être titulaire d'un Master ou diplôme Bac+5 augmente la probabilité d'être transfrontalier de 1,40 point de pourcentage** par rapport aux titulaires d'un CAP/BEP.
- **Être titulaire du CEP augmente la probabilité d'être transfrontalier de 0,77 point de pourcentage** par rapport aux titulaires d'un CAP/BEP.
- **N'avoir aucune scolarité diminue la probabilité d'être transfrontalier de 3,28 points de pourcentage** par rapport aux titulaires d'un CAP/BEP.

### Interprétation économique

### Interprétation économique

La relation entre diplôme et travail transfrontalier est **non linéaire**, avec trois profils distincts :

**Profil 1 — Très diplômés (Master+)** :
- Accès à des postes qualifiés au Luxembourg (finance, audit, consulting, institutions européennes)
- Maîtrise des langues étrangères
- Mobilité internationale facilitée

**Profil 2 — Faiblement qualifiés avec ancienneté (CEP)** :
- Ce résultat reflète probablement un **effet de génération** plutôt qu'un effet du diplôme en soi
- Les titulaires du CEP sont majoritairement des travailleurs âgés (55-65 ans) entrés sur le marché frontalier dans les années 1980-1990, lorsque l'industrie lourde luxembourgeoise et allemande recrutait massivement
- Ces travailleurs ont conservé leur emploi frontalier au fil des décennies, accumulant ancienneté et droits acquis
- Le coefficient capte donc la **persistance** du travail frontalier chez une cohorte spécifique, non un avantage intrinsèque du CEP

**Profil 3 — Qualifications intermédiaires (Bac, BTS) moins mobiles** :
- Ces niveaux trouvent des débouchés locaux satisfaisants (techniciens, employés qualifiés)
- Le différentiel salarial frontalier est moins attractif relativement à leur salaire local
- Moindre incitation à supporter les coûts de la mobilité (trajets, contraintes horaires)

**Profil 4 — Sans qualification ni scolarité** :
- Barrières cumulatives à l'emploi frontalier : langue, mobilité, accès à l'information
- Éloignement des réseaux professionnels transfrontaliers

---

## 7. Conditions de logement et motorisation

> **Variables endogènes** : Les variables de cette section présentent un **risque élevé de causalité inverse**. Les revenus plus élevés des travailleurs transfrontaliers peuvent permettre d'accéder à la propriété, d'habiter en maison individuelle ou de posséder plusieurs véhicules. Les résultats ci-dessous doivent être interprétés comme des **associations statistiques** et non comme des relations causales.

---

## 7. Conditions de logement et motorisation

> **Variables endogènes** : Les variables de cette section présentent un **risque élevé de causalité inverse**. Les revenus plus élevés des travailleurs transfrontaliers peuvent permettre d'accéder à la propriété, d'habiter en maison individuelle ou de posséder plusieurs véhicules. Les résultats ci-dessous doivent être interprétés comme des **associations statistiques** et non comme des relations causales. Un modèle à équations simultanées ou une approche par variables instrumentales serait nécessaire pour identifier les effets causaux.

### Résultats

**Motorisation** (référence : 1 voiture)

| Nombre de voitures   | Association  | Odds Ratio |
|:---------------------|:------------:|:----------:|
| 3 voitures ou plus   | +1,05 pp***  |    1,14    |
| 2 voitures           | +0,74 pp***  |    1,10    |
| Aucune voiture       | -2,04 pp***  |    0,76    |

**Statut d'occupation** (référence : Propriétaire)

| Statut                 | Association  | Odds Ratio |
|:-----------------------|:------------:|:----------:|
| Locataire non-HLM      | -0,51 pp***  |    0,93    |
| Locataire meublé/hôtel | -1,31 pp***  |    0,83    |
| Logé gratuitement      | -1,79 pp***  |    0,78    |
| Locataire HLM          | -3,11 pp***  |    0,64    |

**Type de logement** (référence : Maison individuelle)

| Type de logement     | Association  | Odds Ratio |
|:---------------------|:------------:|:----------:|
| Appartement          | -1,04 pp***  |    0,87    |
| Logement-foyer       | -3,01 pp***  |    0,64    |
| Chambre d'hôtel      | -3,89 pp***  |    0,56    |
| Pièce indépendante   | -3,98 pp***  |    0,55    |

### Interprétation économique

Ces trois dimensions convergent vers un même profil : le travailleur transfrontalier est statistiquement associé à la **propriété en maison individuelle** et à la **multi-motorisation**. Cette configuration peut résulter de trois mécanismes non exclusifs :

1. **Nécessité pratique** : l'automobile est indispensable pour les trajets frontaliers (transports en commun peu développés, horaires décalés). La maison individuelle, plus fréquente en zone périurbaine, offre une localisation optimale entre frontière et bassin de vie.

2. **Effet richesse (causalité inverse)** : les revenus frontaliers élevés — souvent 30 à 50 % supérieurs aux équivalents locaux — permettent l'accession à la propriété et l'achat de véhicules supplémentaires. Dans cette hypothèse, la propriété et la motorisation sont des *conséquences* du travail frontalier, non des *causes*.

3. **Sélection sociale** : les locataires HLM et les ménages sans voiture cumulent généralement des caractéristiques défavorables à l'emploi frontalier (précarité, faible qualification, moindre mobilité). L'association observée refléterait alors une hétérogénéité non observée plutôt qu'un effet direct.

Sans instrumentation adéquate, il est impossible de départager ces mécanismes. Les coefficients de cette section doivent donc être considérés comme **descriptifs** et non comme des leviers d'action pour les politiques publiques.
---

## 8. Temps de travail

> **Variable ambiguë** : Le temps partiel peut être une cause (contrainte de mobilité) ou une conséquence (choix permis par des revenus élevés) du travail transfrontalier.

**Catégorie de référence** : Temps complet

| Variable      | Effet marginal | Odds Ratio |
|----------     |:--------------:|:----------:|
| Temps partiel | +0,77 pp***    | 1,10 |

### Lecture des résultats

- **Travailler à temps partiel est associé à une probabilité supérieure de 0,77 point de pourcentage** par rapport au temps complet.

### Interprétation économique

Ce résultat, contre-intuitif au premier abord, peut s'expliquer par :

1. **Temps partiels choisis** : certains travailleurs frontaliers optent pour un temps partiel afin de concilier trajets longs et vie familiale
2. **Organisation du travail au Luxembourg** : certains emplois frontaliers (notamment dans les services) sont structurés en temps partiel
3. **Effet de composition** : le temps partiel peut refléter des arrangements familiaux où un conjoint travaille à temps partiel pendant que l'autre effectue les trajets frontaliers

** Réserve méthodologique** : Cette variable peut refléter un **arbitrage endogène** entre revenus et qualité de vie.

---

## 9. Inscription dans un établissement d'enseignement

>  **Variable exogène** : Le statut d'étudiant est une caractéristique individuelle préexistante. L'interprétation causale est justifiée.

**Catégorie de référence** : Non inscrit dans un établissement d'enseignement

| Variable | Effet marginal | Odds Ratio |
|----------|:--------------:|:----------:|
| Étudiant | -2,08 pp***    | 0,76 |

### Lecture des résultats

- Toutes choses égales par ailleurs, **être inscrit dans un établissement d'enseignement diminue la probabilité d'être transfrontalier de 2,08 points de pourcentage**.
- **Les chances d'être transfrontalier sont réduites de 24% pour les étudiants** (OR = 0,76).

### Interprétation économique

Les étudiants, même en emploi, sont moins susceptibles d'être transfrontaliers car :

- Leurs emplois sont souvent **temporaires et locaux** (jobs étudiants)
- La **contrainte de présence** aux cours limite la mobilité géographique
- Les **stages frontaliers** sont captés par la variable EMPL_14 (stagiaires rémunérés, effet positif)

---

## 10. Synthèse des déterminants

### 10.1 Facteurs favorisant le travail transfrontalier (variables exogènes)

| Rang | Déterminant                | Effet marginal | Odds Ratio | Interprétation |
|:----:|-------------               |:--------------:|:----------:|----------------|
| 1 | Résider en Moselle            | +15,35 pp      | 4,71 | Proximité Luxembourg |
| 2 | Résider dans le Haut-Rhin     | +13,17 pp      | 3,87 | Proximité Suisse/Allemagne |
| 3 | Résider en Meurthe-et-Moselle | +12,22 pp      | 3,56 | Proximité Luxembourg |
| 4 | Être de nationalité étrangère | +3,12 pp       | 1,56 | Réseaux transnationaux |
| 5 | Être né à l'étranger          | +2,20 pp       | 1,40 | Compétences linguistiques |
| 6 | Doctorat ou Master            | +1,4 à +1,5 pp | 1,19-1,21 | Postes qualifiés |
| 7 | Être ouvrier                  | +0,59 pp       | 1,08 | Demande industrielle |
| 8 | Âge ~40 ans                   | Optimum |   —  | Expérience + mobilité |

### 10.2 Facteurs freinant le travail transfrontalier (variables exogènes)

| Rang | Déterminant                | Effet marginal | Odds Ratio | Interprétation |
|:----:|-------------               |:--------------:|:----------:|----------------|
| 1 | Travailler dans l'agriculture | -6,01 pp       | 0,44 | Ancrage territorial |
| 2 | Secteur public                | -4,16 pp       | 0,55 | Emplois nationaux |
| 3 | Pas de scolarité              | -3,28 pp       | 0,62 | Barrières à l'emploi |
| 4 | Être étudiant                 | -2,08 pp       | 0,76 | Emplois temporaires |
| 5 | Artisan, commerçant           | -1,93 pp       | 0,77 | Clientèle locale |
| 6 | 4+ enfants                    | -1,34 pp       | 0,83 | Contrainte familiale |
| 7 | Être femme                    | -0,79 pp       | 0,88 | Charge familiale |

### 10.3 Variables associées (endogènes ou ambiguës — interprétation prudente)

| Variable            | Effet marginal | Odds Ratio | Risque d'endogénéité |
|----------           |:--------------:|:----------:|----------------------|
| Aucune voiture      | -2,04 pp       | 0,76 | 🔴 Élevé |
| Locataire HLM       | -3,11 pp       | 0,64 | 🔴 Élevé |
| Propriétaire (réf.) | —              | —    | 🔴 Élevé |
| Maison (réf.)       | —              | —    | 🔴 Élevé |
| Temps partiel       | +0,77 pp       | 1,10 | 🟡 Modéré |
| CDI (réf.)          | —              | —    | 🟡 Modéré |

---

## 11. Qualité de l'estimation économétrique

### 11.1 Indicateurs de qualité globale

| Indicateur | Valeur | Interprétation |
|------------|:------:|----------------|
| **Nombre d'observations** | 494 483 | Échantillon très large, puissance statistique élevée |
| **Pseudo R² (McFadden)** | 0,2393 | Bon ajustement pour un modèle binaire sur données individuelles |
| **Log-vraisemblance** | -113 380 | — |
| **Log-vraisemblance nulle** | -149 050 | — |
| **Test LR (χ²)** | 71 340 | — |
| **p-value LR** | < 0,001 | Modèle globalement très significatif |
| **Convergence** | Oui (10 itérations) | Algorithme stable |

### 11.2 Interprétation du Pseudo R²

Un Pseudo R² de McFadden de **0,24** est considéré comme **bon** pour un modèle de choix discret sur données individuelles. À titre de comparaison :

Un Pseudo R² entre 0,20 et 0,40 est généralement jugé satisfaisant en économétrie des choix
Ce critère mesure l'ajustement global mais ne suffit pas à évaluer la capacité prédictive

Les métriques complémentaires présentées ci-dessous (AUC-ROC, Brier Score, validation croisée) confirment que le modèle discrimine efficacement les travailleurs transfrontaliers des non-transfrontaliers et ne souffre pas de sur-ajustement.

### 11.3 Significativité des variables 

| Catégorie                             | Nombre |
|-----------                            |:------:|
| Variables significatives au seuil 1%  | 56 / 65|
| Variables significatives au seuil 5%  | 2 / 65 |
| Variables significatives au seuil 10% | 2 / 65 |
| Variables non significatives          | 5 / 65 |

**Variables non significatives** : GS_1 (agriculteurs exploitants), DEPT_10 (Aube), DEPT_52 (Haute-Marne), NA5_BE (industrie), TYPL_5 (habitation de fortune).

Ces non-significativités sont cohérentes économiquement (effectifs faibles ou absence de différence réelle avec la référence).

### 11.4 Pouvoir discriminant et calibration

La qualité prédictive du modèle est évaluée par des métriques complémentaires au Pseudo R².

L'**AUC-ROC** atteint **0,849** avec un intervalle de confiance bootstrap de [0,848 ; 0,850], ce qui correspond à une excellente capacité discriminante selon les seuils conventionnels (un AUC supérieur à 0,8 est généralement considéré comme très bon). Concrètement, le modèle classe correctement 84,9 % des paires (transfrontalier, non-transfrontalier) tirées au hasard.

Le **Brier Score** s'établit à **0,0681**, contre 0,0815 pour le modèle nul (correspondant à la prévalence de 8,95 %). Le modèle estimé réduit donc l'erreur de prédiction de **16,4 %** par rapport à une prédiction naïve. Le **Brier Skill Score** associé de **0,164** indique un pouvoir prédictif modeste mais significatif.

Enfin, la **calibration par déciles** révèle une adéquation quasi-parfaite entre les probabilités prédites et les fréquences observées. L'écart absolu moyen entre les deux n'est que de **0,0033**, soit 0,33 point de pourcentage. Cela signifie que le modèle ne sur-estime ni ne sous-estime systématiquement les probabilités : un individu auquel le modèle attribue une probabilité de 20 % d'être transfrontalier a effectivement environ 20 % de chances de l'être dans les données observées.

## 11.5 — Validation croisée et stabilité

La robustesse du modèle est testée par validation croisée stratifiée à 5 plis (*5-fold cross-validation*).

L'**AUC moyenne** obtenue sur les 5 plis est de **0,841** avec un écart-type de seulement **0,004**. Cette très faible dispersion entre les plis confirme la stabilité du pouvoir discriminant du modèle. Le **Brier Score** en validation croisée s'établit à **0,069 ± 0,0003**, quasi-identique à l'estimation sur l'échantillon complet.

L'écart-type inter-plis inférieur à **0,5 %** permet de conclure à l'**absence de sur-ajustement**. Le modèle généralise bien à des données non utilisées pour l'estimation, et les coefficients ne sont pas le fruit d'un ajustement excessif aux particularités de l'échantillon.

L'AUC en validation croisée (0,841) reste très proche de l'AUC sur l'échantillon complet (0,849), écartant tout risque d'optimisme excessif dans l'évaluation du modèle. La performance prédictive est donc **reproductible** sur différentes partitions des données.

## 11.6 — Robustesse des coefficients

La stabilité des estimations a été vérifiée selon trois approches complémentaires.

**Exclusion des variables potentiellement endogènes.** L'exclusion des 15 variables présentant un risque de causalité inverse (motorisation, statut d'occupation, type de logement, superficie) produit des coefficients très proches de l'estimation principale. La variation maximale observée est de **4,4 %** sur la variable SEXE_2 (Femme). Les effets géographiques (départements frontaliers) varient de moins de **1 %**, et l'effet de la nationalité étrangère de moins de **2 %**. Cette stabilité confirme que les variables exogènes du modèle capturent des effets **structurels** et non des artefacts liés aux corrélations avec les variables endogènes.

**Estimations par sous-échantillons.** Le modèle a été ré-estimé séparément sur les hommes (Pseudo R² = 0,241, n = 252 847), les femmes (Pseudo R² = 0,233, n = 241 636), les 25-44 ans (Pseudo R² = 0,248, n = 212 156) et les 45-64 ans (Pseudo R² = 0,237, n = 176 892). Les Pseudo R² restent dans une fourchette étroite (0,233 – 0,248), attestant de la **robustesse structurelle** du modèle quel que soit le sous-groupe considéré.

**Comparaison Probit / Logit.** Les effets marginaux issus des spécifications Probit et Logit présentent une corrélation de **0,995**, confirmant que les résultats ne dépendent pas du choix de la fonction de lien.

## 11.7 — Analyse des résidus extrêmes

**4,33 % des observations** (21 415 individus) présentent des résidus de Pearson généralisés supérieurs à 3 en valeur absolue. Ces cas correspondent à des prédictions fortement éloignées de la réalisation.

Ces résidus extrêmes sont concentrés dans les départements frontaliers : la Moselle représente **15,5 %** des cas, le Haut-Rhin **12,2 %**, la Meurthe-et-Moselle **11,6 %** et le Bas-Rhin **9,8 %**. Cette répartition géographique n'est pas le fruit du hasard : elle reflète la forte hétérogénéité intra-départementale des comportements dans les zones proches des frontières.

Ces observations ne constituent **pas des erreurs de mesure** ni des anomalies à exclure. Elles reflètent l'**hétérogénéité réelle** de la population. Les faux négatifs correspondent à des individus résidant en zone frontalière mais travaillant en France, pour des raisons de préférences personnelles, d'emploi local attractif ou de contraintes familiales. Les faux positifs correspondent à des individus résidant loin des frontières mais travaillant néanmoins à l'étranger, souvent liés à des situations professionnelles spécifiques.

La concentration des résidus extrêmes dans les départements frontaliers illustre les **limites intrinsèques** d'un modèle probabiliste : même avec des caractéristiques observables identiques, les choix individuels demeurent partiellement irréductibles aux variables du modèle.

---

## 12. Limites méthodologiques et extensions futures

### 12.1 Limites du modèle actuel

#### Endogénéité de certaines variables

Plusieurs variables explicatives (propriété, motorisation, type de logement) sont potentiellement **endogènes** : elles peuvent être des conséquences plutôt que des causes du travail transfrontalier. Sans correction, les coefficients estimés sont **biaisés** et ne permettent pas d'inférence causale.

#### Imperfections de la forme fonctionnelle

Les tests de spécification classiques (Link test de Pregibon, Hosmer-Lemeshow) rejettent l'hypothèse nulle, suggérant que la forme fonctionnelle du modèle Probit n'est pas parfaitement adaptée aux données. Bien que ces signaux soient mécaniquement amplifiés par la taille de l'échantillon (près de 500 000 observations), ils rappellent que la spécification retenue demeure une **approximation** de la réalité. En particulier, la concentration géographique extrême du phénomène — 90 % des transfrontaliers dans 4 départements — pourrait justifier des modélisations alternatives intégrant une structure spatiale explicite.

#### Variables omises

Le modèle n'inclut pas :
- La **distance précise à la frontière** (seul le département est disponible)
- Les **compétences linguistiques** (allemand, luxembourgeois)
- Le **différentiel salarial individuel** entre emploi local et frontalier
- Les **préférences individuelles** (aversion aux trajets, attachement territorial)

#### Hétérogénéité non observée

Des caractéristiques individuelles non mesurées (motivation, réseau professionnel, santé) peuvent influencer simultanément le choix de résidence et le travail frontalier.

#### Coupe transversale

L'analyse sur une seule année (2022) ne permet pas d'étudier les dynamiques temporelles (entrées/sorties du travail frontalier, effets de la conjoncture).

---

### 12.2 Extensions futures recommandées

#### 1. Traitement de l'endogénéité

| Méthode | Principe | Application possible |
|---------|----------|----------------------|
| **Probit à Variables Instrumentales (IV Probit)** | Trouver un instrument Z affectant la variable endogène mais pas Y directement | Instrument potentiel : taux de propriétaires dans la commune d'origine |
| **Control Function Approach** | Inclure les résidus de la 1ère étape dans le probit | Permet de tester l'endogénéité |
| **Probit bivarié récursif** | Modéliser simultanément P(propriétaire) et P(transfrontalier) | Identification par restrictions d'exclusion |

#### 2. Modèle Probit avec interactions par genre (prévu)

Objectif : Tester si les déterminants ont des **effets différenciés** selon le sexe.

Spécification :
$$P(Y_i = 1) = \Phi(\beta_0 + \beta_1 Femme_i + \beta_2 X_i + \beta_3 (Femme_i \times X_i))$$

Permettra de répondre à : *L'effet des enfants est-il plus fort pour les femmes ?*

#### 3. Modèle Probit bivarié pour les couples (prévu)

Objectif : Tester la **coordination des décisions** au sein des couples.

Spécification :
$$Y_{1i}^* = X_{1i}'\beta_1 + \varepsilon_{1i}$$
$$Y_{2i}^* = X_{2i}'\beta_2 + \varepsilon_{2i}$$

avec $\text{Corr}(\varepsilon_1, \varepsilon_2) = \rho$

Permettra de répondre à : *Les choix professionnels des conjoints sont-ils coordonnés ?*


#### 5. Extensions de données

- **Panel** : Suivre les individus sur plusieurs années pour identifier les transitions vers le travail frontalier
- **Variables géographiques fines** : Distance à la frontière, accessibilité routière
- **Variables fiscales** : Revenus individuels pour calculer le différentiel salarial

---

## 13. Conclusion

Cette analyse économétrique met en évidence une **hiérarchie claire des déterminants** du travail transfrontalier dans le Grand Est :

1. **La géographie domine** : résider dans un département frontalier (Moselle, Haut-Rhin, Meurthe-et-Moselle) est le facteur le plus déterminant, avec des effets marginaux de 12 à 15 points de pourcentage et des odds ratios de 3,5 à 4,7.

2. **Les réseaux transnationaux comptent** : la nationalité étrangère (+3,1 pp) et la naissance à l'étranger (+2,2 pp) facilitent l'accès au travail frontalier, probablement via les compétences linguistiques et les connexions sociales.

3. **Le profil socioprofessionnel sélectionne** : le travail frontalier concerne prioritairement les ouvriers en CDI (+0,6 pp) et les cadres très diplômés (+1,4 à +1,5 pp), tout en excluant les indépendants (-3,6 pp), le secteur public (-4,2 pp) et les formes d'emploi précaires.

4. **La famille contraint** : chaque enfant supplémentaire réduit la probabilité d'être transfrontalier de 0,3 à 0,4 point de pourcentage, avec une accélération au-delà de deux enfants.

5. **Le genre et l'âge structurent les comportements** : les femmes présentent une probabilité inférieure de 0,8 point de pourcentage, toutes choses égales par ailleurs. La probabilité suit une courbe en U inversé avec un maximum autour de 41 ans, reflétant l'arbitrage entre accumulation d'expérience et pénibilité croissante des trajets.

6. **Les variables de logement et motorisation sont associées** au travail frontalier (propriété, maison individuelle, multi-motorisation), mais le sens de la causalité reste incertain : ces caractéristiques peuvent être des conséquences des revenus frontaliers plutôt que des causes.

7. **Le modèle est statistiquement robuste** : avec une AUC-ROC de 0,849 et une calibration quasi-parfaite (écart moyen de 0,33 point de pourcentage), le modèle discrimine efficacement les profils transfrontaliers. La validation croisée à 5 plis (AUC = 0,841 ± 0,004) confirme l'absence de sur-ajustement. Les coefficients demeurent stables lors de l'exclusion des variables potentiellement endogènes.

Ces résultats fournissent une base solide pour les extensions prévues : le **probit avec interactions par genre** permettra d'identifier si les déterminants (notamment la présence d'enfants) affectent différemment les hommes et les femmes ; le **probit bivarié** testera la coordination des décisions professionnelles au sein des couples.


Ces résultats fournissent une base solide pour l'estimation des modèles complémentaires prévus (probit avec interactions par genre, probit bivarié pour les décisions de couple) et pour l'élaboration de recommandations en matière de politique publique (transport, garde d'enfants, formation linguistique).

---

*Rapport rédigé le 7 janvier 2026 — Projet INSEE Mobilité Transfrontalière Grand Est
*Romain Mehdi FEHRI
*Master 2 Statistique, Économétrie & Data Science — Université de Strasbourg*